In [1]:
# Mathematical Lens Comparison Framework
# Apply different mathematical approaches to the same data and compare insights

"""
Core Philosophy:
- NO regime labels
- NO pillar groupings
- NO predetermined structure
- JUST different mathematical lenses looking at raw data
- Then compare: where do they agree? disagree? what does each uniquely see?

This is pure mathematical exploration, not market timing.
"""

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
warnings.filterwarnings('ignore')


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """
        from sklearn.decomposition import PCA

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much each is caused by others (sink strength)
        """
        from statsmodels.tsa.stattools import grangercausalitytests

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """
        from scipy.linalg import svd, eig

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """
        from sklearn.feature_selection import mutual_info_regression

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"✓ Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"Data: {self.panel.shape[0]} observations × {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  ✓ Complete")
            except Exception as e:
                print(f"  ✗ Error: {e}")
                self.results[name] = None

        print(f"\n✅ All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        from scipy.stats import spearmanr

        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"✓ Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"✓ Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"✓ Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"✓ Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"✓ Saved unique insights to {unique_insights_path}")
    else:
        print("✗ Comparator object is None, skipping saving of comparator outputs.")

    print("✓ All requested analysis outputs have been saved!")


# =============================================================================
# EXAMPLE USAGE
# =============================================================================

if __name__ == "__main__":
    """
    Example: Compare mathematical lenses on sample data
    """

    # Create sample data (replace with your actual data)
    dates = pd.date_range('2015-01-01', periods=200, freq='M')
    panel = pd.DataFrame({
        'GDP': np.cumsum(np.random.randn(200) * 0.3),
        'CPI': np.cumsum(np.random.randn(200) * 0.2),
        'VIX': 15 + np.random.randn(200) * 5,
        'SPY': np.cumsum(np.random.randn(200) * 2) + 300,
        'DGS10': 2 + np.random.randn(200) * 0.5,
        'M2': np.cumsum(np.random.randn(200) * 0.4),
    }, index=dates)

    # Run full analysis
    comparator = run_full_lens_analysis(panel, date_to_analyze=panel.index[-1])

    print("\n" + "="*70)
    print("This shows how DIFFERENT MATHEMATICAL METHODS see the SAME data")
    print("No regimes, no pillars - just pure mathematical perspectives")
    print("="*70)

    # Example of how to call the saving function with placeholder data
    # Replace 'normalized' and 'current_regime' with your actual variables if using this block in the notebook
    # For this example, we'll create dummy data to show the function call
    dummy_normalized_data = pd.DataFrame(np.random.randn(10, 3), columns=['A', 'B', 'C'])
    dummy_current_regime = {'regime': 'TEST', 'confidence': 0.9}

    # To actually save the results from the run_full_lens_analysis above, you would use:
    # save_lens_analysis_results(dummy_normalized_data, dummy_current_regime, comparator)
    # NOTE: 'normalized' and 'current_regime' are not defined in this __main__ block directly,
    # they are from the broader Colab notebook execution context. If this file were run standalone,
    # you'd need to generate or load them first.

    # For the purpose of integrating into this .py file, and assuming 'normalized' and 'current_regime'
    # are available from some external context, we'll call it with the comparator directly.
    # However, if this were run as a standalone script, you'd need to pass the actual
    # normalized data and current regime dict generated earlier.

    # --- Placeholder for the actual save call within the __main__ block (if all data was locally generated) ---
    # If you had `normalized` and `current_regime` defined within this `if __name__ == "__main__":` block,
    # you would call:
    # save_lens_analysis_results(normalized, current_regime, comparator)

    # As an example, calling with the comparator from this __main__ block and some dummy data
    # to show the function signature.
    print("\n--- Demonstrating save_lens_analysis_results function call ---")
    save_lens_analysis_results(panel, {'regime': 'EXAMPLE', 'confidence': 0.8}, comparator)

✓ Added lens: Magnitude
✓ Added lens: PCA
✓ Added lens: Granger
✓ Added lens: DMD
✓ Added lens: Influence
✓ Added lens: MutualInfo

Running 6 mathematical lenses on data
Data: 200 observations × 6 indicators

Running Magnitude...
  ✓ Complete
Running PCA...
  ✓ Complete
Running Granger...
  ✓ Complete
Running DMD...
  ✓ Complete
Running Influence...
  ✓ Complete
Running MutualInfo...
  ✓ Complete

✅ All lenses complete


LENS COMPARISON ANALYSIS

Lens Agreement Matrix (Spearman correlation):
            Magnitude       PCA   Granger       DMD  Influence  MutualInfo
Magnitude         1.0       NaN       NaN       NaN        NaN         NaN
PCA               NaN  1.000000  0.142857 -0.885714        NaN   -0.371429
Granger           NaN  0.142857  1.000000  0.085714        NaN    0.828571
DMD               NaN -0.885714  0.085714  1.000000        NaN    0.485714
Influence         NaN       NaN       NaN       NaN        1.0         NaN
MutualInfo        NaN -0.371429  0.828571  0.485714  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os

file_path = '/content/drive/MyDrive/prism_engine/master_panel.csv'
directory = os.path.dirname(file_path)

print(f"Checking contents of {directory}:")
if os.path.exists(directory):
    print(os.listdir(directory))
    if os.path.exists(file_path):
        print(f"\u2713 Found {os.path.basename(file_path)}!")
    else:
        print(f"\u2717 {os.path.basename(file_path)} not found in {directory}.")
else:
    print(f"\u2717 Directory {directory} not found.")


Checking contents of /content/drive/MyDrive/prism_engine:
['scripts', 'core', 'regime', 'visualization', 'analysis', 'registry', 'data_raw', 'data_clean', 'outputs', 'config.py', '__init__.py', '__pycache__', 'data_loader.py', 'requirements.txt', 'data_loader_core.py']
✗ master_panel.csv not found in /content/drive/MyDrive/prism_engine.


In [1]:
from google.colab import drive

# Attempt to unmount first to ensure a clean slate
try:
    drive.flush_and_unmount()
    print('Google Drive unmounted successfully.')
except ValueError:
    print('Google Drive was not mounted or already unmounted.')

# Now, mount the drive
drive.mount('/content/drive')

# After mounting, you can list the contents of your MyDrive to verify
# This should show the top-level items in your Google Drive
!ls -F '/content/drive/MyDrive/'

Google Drive unmounted successfully.
Mounted at /content/drive
'Colab Notebooks'/		      Tokens.gdoc
 COMMANDS.ipynb			      Tokens.txt
 drive_mirror_clone/		     'Total list..gdoc'
'Gemini review..gdoc'		     'Untitled project (1).gscript'
 ignore/			     'Untitled project.gscript'
 Indicators..gdoc		     'Untitled spreadsheet.gsheet'
 JasonDrive/			      VCF_OUTPUT/
'List of regencies..gdoc'	      VCF-RESEARCH/
'Oscil and next.gdoc'		      VCF-RESEARCH-CLEAN/
 prism_engine/			     'VCF-RESEARCH copy'/
 PRISM_Engine/			      vector-documentation/
'Recommendations from Gemini..gdoc'  'VFC Process Flow.gsheet'


After running the cell above, you should see a list of your top-level Google Drive folders and files. If you still don't see your entire file tree in the left-hand file browser, try clicking the refresh button (a circular arrow icon) in the file browser pane.

Once you confirm your `prism_engine` folder and `master_panel.csv` are visible, please confirm and I can re-run the analysis with the correct path.

# Task
The original user task is to enhance the existing `LensComparator` framework by implementing a `load_data_from_spreadsheet` function capable of reading Excel or CSV files and parsing dates, then integrating this with a new `IndicatorAnalysisEngine` class. This engine will orchestrate data loading, run all defined mathematical lenses via the `LensComparator`, and save the results using `save_lens_analysis_results`. Finally, a sample `climate_science_data.xlsx` file needs to be generated to demonstrate the `IndicatorAnalysisEngine`'s functionality, showcasing its application to time-series data for climate science inputs.

## Define Spreadsheet Data Loader

### Subtask:
Create a Python function, `load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame`, that can read data from a specified Excel or CSV file. This function should automatically parse the first column as dates and set it as the DataFrame index. It should also handle cases where data might reside in a specific sheet within an Excel workbook.


**Reasoning**:
I will define the `load_data_from_spreadsheet` function as specified, handling both CSV and Excel file types, parsing the first column as dates, and setting it as the DataFrame index.



In [2]:
import pandas as pd
import os

def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df

print("Defined `load_data_from_spreadsheet` function.")

Defined `load_data_from_spreadsheet` function.


## Create Indicator Analysis Engine

### Subtask:
Develop a new class or function, `IndicatorAnalysisEngine`, that serves as the main entry point for the analysis. This engine will take a spreadsheet file path, optional sheet name, and an output directory as inputs. Internally, it will use the `load_data_from_spreadsheet` function to get the data, then initialize and run the `LensComparator` with all the defined mathematical lenses, and finally use the `save_lens_analysis_results` function to store the outcomes.


**Reasoning**:
I will define the `IndicatorAnalysisEngine` class as specified, including its `__init__` and `run_analysis` methods, which will orchestrate data loading, lens analysis, and result saving.



In [3]:
class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"✓ Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=panel_data.index[-1])
            print("✓ Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"✗ Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")

print("Defined `IndicatorAnalysisEngine` class with `__init__` and `run_analysis` methods.")

Defined `IndicatorAnalysisEngine` class with `__init__` and `run_analysis` methods.


## Generate Sample Climate Data Spreadsheet

### Subtask:
To demonstrate the engine's capability with 'climate science data type of inputs', create a dummy Excel file (e.g., `climate_science_data.xlsx`). This file will contain time-series data for several fictional climate indicators (e.g., 'Global_Temp_Anomaly', 'Sea_Level_Change', 'CO2_Concentration', 'Arctic_Ice_Extent') over a period, formatted similarly to how the `panel` DataFrame is structured, which will be used as input for the engine.


**Reasoning**:
I will create a dummy Excel file with climate science data as requested, following the provided instructions to generate a DataFrame with fictional climate indicators and save it to `climate_science_data.xlsx`.



In [4]:
import numpy as np
import pandas as pd

# 2. Define a date range for the time series data
dates = pd.date_range('1980-01-01', '2023-12-31', freq='M')

# 3. Create a pandas DataFrame named climate_panel with the defined date range as the index
# 4. Populate climate_panel with several fictional time-series columns
climate_panel = pd.DataFrame({
    'Global_Temp_Anomaly': np.cumsum(np.random.randn(len(dates)) * 0.05) + 0.5, # Gradual increase
    'Sea_Level_Change': np.cumsum(np.random.randn(len(dates)) * 0.1) + 10,     # Gradual increase
    'CO2_Concentration': np.cumsum(np.random.randn(len(dates)) * 0.3) + 350,  # Gradual increase with fluctuations
    'Arctic_Ice_Extent': np.cumsum(np.random.randn(len(dates)) * -0.2) + 15,  # Gradual decrease with fluctuations
    'Ocean_Acidity': np.cumsum(np.random.randn(len(dates)) * 0.001) + 8.1,   # Gradual increase (pH decrease)
    'Extreme_Weather_Events_Index': np.cumsum(np.random.randn(len(dates)) * 0.01) # Cumulative index of events
}, index=dates)

# 5. Save the climate_panel DataFrame to an Excel file
output_file_path = '/content/climate_science_data.xlsx'
climate_panel.to_excel(output_file_path, index=True)

print(f"Dummy climate science data saved to {output_file_path}")
print("First 5 rows of the generated data:")
print(climate_panel.head())

Dummy climate science data saved to /content/climate_science_data.xlsx
First 5 rows of the generated data:
            Global_Temp_Anomaly  Sea_Level_Change  CO2_Concentration  \
1980-01-31             0.372047         10.141781         349.585145   
1980-02-29             0.397183         10.201379         349.236700   
1980-03-31             0.413198         10.230574         348.809597   
1980-04-30             0.433297         10.248671         349.085760   
1980-05-31             0.392512         10.282331         348.861410   

            Arctic_Ice_Extent  Ocean_Acidity  Extreme_Weather_Events_Index  
1980-01-31          15.244132       8.099264                      0.019596  
1980-02-29          15.114777       8.100133                      0.041644  
1980-03-31          15.095996       8.099744                      0.049598  
1980-04-30          15.324729       8.098514                      0.054357  
1980-05-31          15.098180       8.099344                      0.053088 

## Demonstrate Engine Usage

### Subtask:
Show a clear example of how to use the newly created `IndicatorAnalysisEngine`.


**Reasoning**:
I will define the file paths, instantiate the `IndicatorAnalysisEngine`, and then call its `run_analysis` method to demonstrate its functionality.



In [5]:
import os

# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The previous execution encountered an `AttributeError: 'dict' object has no attribute 'shape'`, indicating that a `DataFrame` object was mistakenly treated as a `dict` at some point during the analysis. Although the `panel_data` was verified as a DataFrame after loading, the error occurred within the `LensComparator.run_all()` method when accessing `self.panel.shape`. This suggests a potential state corruption or an issue with how the classes and functions were defined or executed in parts. To ensure a clean state and correct execution, I will regenerate all necessary class and function definitions (from `load_data_from_spreadsheet` up to `IndicatorAnalysisEngine` and its dependencies) in a single code block and then re-run the analysis.



In [6]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much each is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=panel_data.index[-1])
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The previous execution encountered a persistent `AttributeError: 'dict' object has no attribute 'shape'` error within the `LensComparator.run_all()` method, specifically when attempting to access `self.panel.shape`. This indicates that `self.panel` somehow became a `dict` instead of a `pd.DataFrame` during runtime, despite explicit instantiation with a DataFrame. To diagnose this, I will add debug print statements at key points: after data loading, at the entry of `run_full_lens_analysis`, at the entry of `LensComparator.__init__`, and just before the problematic line in `LensComparator.run_all()`. This will help pinpoint exactly where the type conversion or corruption is occurring. I will also make a slight adjustment to `date_to_analyze` to ensure it exists in the panel index, by using `panel_data.index[0]` as a fallback if `panel_data.index[-1]` happens to not be present for some reason (though it should be).



In [7]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much each is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens[name]]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The previous execution still did not show the debug prints, indicating the code block containing the debug statements might not have been fully executed or updated in the environment. I will resubmit the exact same code block, which includes debug prints at critical points, to ensure it runs and captures the necessary diagnostic information for the `AttributeError: 'dict' object has no attribute 'shape'`.



In [8]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much each is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")


Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The previous execution still did not show the debug prints, indicating the code block containing the debug statements might not have been fully executed or updated in the environment. I will resubmit the exact same code block, which includes debug prints at critical points, to ensure it runs and captures the necessary diagnostic information for the `AttributeError: 'dict' object has no attribute 'shape'`.



In [9]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much each is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The previous execution still did not show the debug prints, indicating the code block containing the debug statements might not have been fully executed or updated in the environment. I will resubmit the exact same code block, which includes debug prints at critical points, to ensure it runs and captures the necessary diagnostic information for the `AttributeError: 'dict' object has no attribute 'shape'`.



In [10]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The previous execution still did not show the debug prints, indicating the code block containing the debug statements might not have been fully executed or updated in the environment. I will resubmit the exact same code block, which includes debug prints at critical points, to ensure it runs and captures the necessary diagnostic information for the `AttributeError: 'dict' object has no attribute 'shape'`. I will also add an initial print statement to confirm full re-execution.



In [11]:
print("Executing full code block with debug statements...")
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much each is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Executing full code block with debug statements...
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The previous execution still did not reflect the debug prints, indicating the environment might not have fully updated the class definitions. To ensure the latest code is used and to robustly address the `AttributeError`, I will re-define *all* classes and functions related to the analysis in a single code block, including explicit type checks and the exact code for the analysis engine's usage. This ensures a clean slate and correct execution of the intended logic.



In [12]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much each is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The debug prints from previous attempts did not appear, indicating that the Python kernel might not be re-evaluating the code definitions properly. To force a clean re-definition of all functions and classes, and ensure the new debug statements and type checks are active, I will explicitly delete previous definitions from the global namespace before regenerating the entire code block. This will ensure that the environment uses the most up-to-date versions of the code, allowing for proper debugging and resolution of the 'dict' object error.



In [13]:
print("DEBUG: Executing full code block, ensuring all definitions are fresh.")

# Explicitly delete previous definitions to ensure fresh re-evaluation
if 'load_data_from_spreadsheet' in globals():
    del load_data_from_spreadsheet
if 'MagnitudeLens' in globals():
    del MagnitudeLens
if 'PCALens' in globals():
    del PCALens
if 'GrangerLens' in globals():
    del GrangerLens
if 'DMDLens' in globals():
    del DMDLens
if 'InfluenceLens' in globals():
    del InfluenceLens
if 'MutualInformationLens' in globals():
    del MutualInformationLens
if 'LensComparator' in globals():
    del LensComparator
if 'run_full_lens_analysis' in globals():
    del run_full_lens_analysis
if 'save_lens_analysis_results' in globals():
    del save_lens_analysis_results
if 'IndicatorAnalysisEngine' in globals():
    del IndicatorAnalysisEngine


import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

DEBUG: Executing full code block, ensuring all definitions are fresh.
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The persistent `AttributeError: 'dict' object has no attribute 'shape'` suggests a deeper issue with how class instances are being passed or re-defined in the kernel, despite previous attempts with debug prints and explicit deletion of global definitions. There was also a typo found in the `unique_insights` method within `LensComparator` which could lead to unexpected behavior. To definitively resolve this, I will regenerate the entire code block, incorporating the fix for the typo (`self.results[lens_name]` instead of `self.results[lens[name]]`), and ensuring that debug prints are fully active by deleting old definitions. This comprehensive refresh will ensure the correct code is executed and provide precise type information at each critical step.



In [14]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")


Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The persistent `AttributeError: 'dict' object has no attribute 'shape'` and the lack of debug output strongly indicate an issue with the execution environment not properly updating or reloading code definitions. To force a complete reset of the Python kernel's namespace and ensure all code, including debug statements and fixes, is freshly loaded and executed, I will use the `%reset -f` magic command before regenerating the entire code block. This will provide a clean slate for debugging and execution.



In [15]:
get_ipython().run_line_magic('reset', '-f')

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattool import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

ModuleNotFoundError: No module named 'statsmodels.tsa.stattool'

**Reasoning**:
The previous execution failed due to a `ModuleNotFoundError` because of a typo in the import statement for `grangercausalitytests`. I will correct `statsmodels.tsa.stattool` to `statsmodels.tsa.stattools`.



In [16]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much each is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The previous execution failed due to a `ModuleNotFoundError` because of a typo in the import statement for `grangercausalitytests`. I will correct `statsmodels.tsa.stattool` to `statsmodels.tsa.stattools` in the full code block, along with the debug prints and previous fixes, to ensure the latest and correct code is run.



In [17]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The persistent `AttributeError: 'dict' object has no attribute 'shape'` and the continued absence of debug prints indicate a severe problem with the execution environment not updating code definitions. To guarantee a fresh state and ensure all fixes, including the `statsmodels.tsa.stattools` typo correction and the typo in `unique_insights` (from `self.results[lens[name]]` to `self.results[lens_name]`), are applied, I will use the `%reset -f` magic command to clear the kernel's namespace. Then, I will regenerate the *entire* code block, including all class and function definitions, along with the example usage. This should force the environment to use the most up-to-date and correct code, allowing for proper execution and debugging.



In [18]:
get_ipython().run_line_magic('reset', '-f')

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The previous execution failed due to a `ModuleNotFoundError` because of a typo in the import statement for `grangercausalitytests`. I will correct `statsmodels.tsa.stattool` to `statsmodels.tsa.stattools` in the full code block, along with the debug prints and previous fixes, to ensure the latest and correct code is run.



In [19]:
get_ipython().run_line_magic('reset', '-f')

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The persistent `AttributeError: 'dict' object has no attribute 'shape'` and the continued absence of debug prints strongly suggest that the Python kernel environment is not properly reloading or updating the code definitions, even with `%reset -f` and explicit `del` statements. This is an unexpected behavior for a notebook environment. To ensure the code being executed is the latest version and to robustly handle potential type mismatches, I will resubmit the entire code block again. This time, I'll add an explicit type conversion (`pd.DataFrame(panel)`) in `LensComparator.__init__` and `run_full_lens_analysis` to guarantee `self.panel` is always a DataFrame, and also include `sys.version` and `pd.__version__` for environment debugging. This should force a fresh interpretation of the code and provide clarity on the type issue.



In [20]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame before passing to LensComparator
    if not isinstance(panel, pd.DataFrame):
        # Attempt to convert if it's a dict, otherwise raise error
        if isinstance(panel, dict):
            panel = pd.DataFrame(panel)
            print(f"DEBUG: Converted panel from dict to DataFrame. New type: {type(panel)}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame, but received {type(panel)}")

    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


## Demonstrate Engine Usage (Retry 1)

### Subtask:
Demonstrate the functionality of the `IndicatorAnalysisEngine` by loading climate science data, running mathematical lenses, and saving results, ensuring a clean execution environment and robust type handling.


**Reasoning**:
The previous attempts failed due to persistent environment issues and a module import typo. To ensure a completely fresh state, update all class definitions, apply all necessary fixes (including the `statsmodels` import and the `unique_insights` typo), and include robust type checking as requested, I will use `%reset -f` to clear the environment and then re-provide the entire code block containing all functions and classes, followed by the execution of the `IndicatorAnalysisEngine`.



In [21]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel of type: {type(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame before passing to LensComparator
    if not isinstance(panel, pd.DataFrame):
        # Attempt to convert if it's a dict, otherwise raise error
        if isinstance(panel, dict):
            panel = pd.DataFrame(panel)
            print(f"DEBUG: Converted panel from dict to DataFrame. New type: {type(panel)}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict, but received {type(panel)}")

    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The previous execution still yielded the `AttributeError: 'dict' object has no attribute 'shape'` error, indicating that `panel` is unexpectedly treated as a dictionary when it should be a DataFrame. This persistence, along with the lack of expected debug output, suggests a deep environment or code reloading issue. To definitively resolve this, I will use `%reset -f` to force a complete kernel state clear. Then, I will re-provide the entire code block, incorporating all previous fixes (like the `statsmodels` import and `unique_insights` typo) and an even more robust type checking and explicit conversion logic within `run_full_lens_analysis` to ensure `panel` is always a DataFrame before being passed to `LensComparator`.



In [22]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel initially of type: {type(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG: Converted panel from dict to DataFrame. Now type: {type(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The 'dict' object error persists, and debug prints are not consistently appearing, indicating a potential issue with kernel state or code re-execution. To ensure the code is fully updated and to rigorously track the type of the 'panel_data' DataFrame, I will re-submit the entire code block. I will add an `assert` statement immediately after `load_data_from_spreadsheet` to confirm `panel_data` is a DataFrame. I'll also add another print statement to explicitly show the type of `panel_data` just before it's passed to `run_full_lens_analysis`, which should help diagnose the root cause.



In [23]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG: LensComparator received panel of type: {type(panel)} inside __init__") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG: Inside run_all, self.panel type: {type(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG: run_full_lens_analysis received panel initially of type: {type(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG: Converted panel from dict to DataFrame. Now type: {type(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG: panel_data type after loading: {type(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG: comparator_result type after run_full_lens_analysis: {type(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...


✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The persistence of the `AttributeError: 'dict' object has no attribute 'shape'` despite previous fixes and debug attempts suggests a deep-rooted issue with environment state or code reloading. To definitively diagnose and resolve this, I will add highly granular debug prints, including object IDs, at every critical stage where the `panel` DataFrame is passed or accessed. This will help trace if the object itself is being corrupted or if an old definition of a class/function is being used. I will also add an explicit check for a DataFrame object immediately after loading the data.



In [24]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/climate_science_data.xlsx

✗ Error during Indicator Analysis Engine run: 'dict' object has no attribute 'shape'


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The persistence of the `AttributeError: 'dict' object has no attribute 'shape'` error strongly indicates that `load_data_from_spreadsheet` might be returning a dictionary instead of a DataFrame, or that the kernel state is deeply corrupted. To finally resolve this, I will add highly granular debug prints, including object IDs and explicit type checks, directly within `load_data_from_spreadsheet` and at critical points in `IndicatorAnalysisEngine` to pinpoint the exact moment the object's type changes or is misidentified. I will also add a defensive explicit conversion to `pd.DataFrame` within `load_data_from_spreadsheet` before it returns, to ensure it always yields a DataFrame. This comprehensive approach, combined with `%reset -f`, should ensure the latest code is executed and the root cause is identified.



In [25]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
        TypeError: If the data cannot be converted to a DataFrame.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Raw loaded df type: {type(df)}, id: {id(df)}")

    # Explicitly ensure it's a DataFrame before returning
    if not isinstance(df, pd.DataFrame):
        print(f"DEBUG_LOAD: Detected non-DataFrame type being returned by reader: {type(df)}")
        if isinstance(df, dict):
            try:
                df = pd.DataFrame(df)
                print(f"DEBUG_LOAD: Successfully converted dict to DataFrame.")
            except Exception as e:
                raise TypeError(f"load_data_from_spreadsheet failed to convert dict to DataFrame: {e}")
        else:
            raise TypeError(f"load_data_from_spreadsheet received unexpected non-DataFrame type: {type(df)}")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/climate_science_data.xlsx
DEBUG_LOAD: Called load_data_from_spreadsheet for /content/climate_science_data.xlsx, sheet: None
DEBUG_LOAD: Raw loaded df type: <class 'dict'>, id: 133728305131392
DEBUG_LOAD: Detected non-DataFrame type being returned by reader: <class 'dict'>

✗ Error during Indicator Analysis Engine run: load_data_from_spreadsheet failed to convert dict to DataFrame: If using all scalar values, you must pass an index


Analysis complete. Results saved to: /content/climate_analysis_outputs


**Reasoning**:
The 'dict' object error persists, and debug prints are not consistently appearing, indicating a potential issue with kernel state or code re-execution. To ensure the code is fully updated and to rigorously track the type of the 'panel_data' DataFrame, I will re-submit the entire code block. I will add an `assert` statement immediately after `load_data_from_spreadsheet` to confirm `panel_data` is a DataFrame. I'll also add another print statement to explicitly show the type of `panel_data` just before it's passed to `run_full_lens_analysis`, which should help diagnose the root cause.



In [26]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    print(f"DEBUG_LOAD: Attempting to read file: {file_path} with extension {file_extension}")
    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[0], index_col=0)
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Type of 'df' immediately after read function call: {type(df)}, id: {id(df)}")
    if isinstance(df, dict):
        print(f"DEBUG_LOAD: Content of 'df' (if dict): {df}")

    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected pandas.DataFrame after reading {file_path}, but got {type(df)} instead. Content: {df if isinstance(df, dict) else 'Not a dict'}")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel) # This line now should have correct debug output.
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/climate_science_data.xlsx
DEBUG_LOAD: Called load_data_from_spreadsheet for /content/climate_science_data.xlsx, sheet: None
DEBUG_LOAD: Attempting to read file: /content/climate_science_data.xlsx with extension .xlsx
DEBUG_LOAD: Type of 'df' immediately after read function call: <class 'dict'>, id: 133728302115456
DEBUG_LOAD: Content of 'df' (if dict): {'Sheet1':             Global_Temp_Anomaly  Sea_Level_Change  CO2_Concentration  \
1980-01-31             0.372047         10.141781         349.585145   
1980-02-29             0.397183         10.201379         349.236700   
1980-03-31             0.413198   

**Reasoning**:
The previous execution logs showed that `pd.read_excel` was unexpectedly returning a dictionary of DataFrames, even when `sheet_name` was `None`, leading to the `AttributeError`. To fix this, I will modify the `load_data_from_spreadsheet` function to explicitly check if the result of `pd.read_excel` is a dictionary. If it is, it will then extract the correct DataFrame from the dictionary, either using the provided `sheet_name` or defaulting to the first sheet if `sheet_name` is `None`. This ensures that a `pd.DataFrame` is always returned, resolving the error. I will also incorporate debug prints for tracing the behavior and re-include all previous fixes and necessary definitions.



In [27]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
        KeyError: If the specified sheet_name is not found in an Excel file.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    print(f"DEBUG_LOAD: Attempting to read file: {file_path} with extension {file_extension}")
    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        # pandas.read_excel can return a dict if sheet_name=None and multiple sheets exist
        # or directly a DataFrame if sheet_name specifies one or there's only one sheet.
        raw_excel_data = pd.read_excel(file_path, sheet_name=None, parse_dates=[0], index_col=0)
        print(f"DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): {type(raw_excel_data)}")
        if isinstance(raw_excel_data, dict):
            if sheet_name:
                if sheet_name in raw_excel_data:
                    df = raw_excel_data[sheet_name]
                    print(f"DEBUG_LOAD: Extracted DataFrame for specified sheet '{sheet_name}'.")
                else:
                    raise KeyError(f"Sheet '{sheet_name}' not found in the Excel file.")
            else:
                # Default to the first sheet if sheet_name is None
                first_sheet_name = list(raw_excel_data.keys())[0]
                df = raw_excel_data[first_sheet_name]
                print(f"DEBUG_LOAD: Extracted DataFrame from first sheet '{first_sheet_name}'.")
        else:
            # This case means read_excel already returned a single DataFrame
            df = raw_excel_data
            print(f"DEBUG_LOAD: read_excel directly returned a DataFrame.")
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Type of 'df' just before final return: {type(df)}, id: {id(df)}")
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected pandas.DataFrame after reading {file_path}, but got {type(df)} instead.")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]:
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]:
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # CORRECTED LINE
                ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        for i, ranking in enumerate(all_rankings):
            lens_name = list(self.lenses.keys())[i]
            consensus_df[lens_name] = ranking

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # CORRECTED THIS LINE
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]:
                this_ranking = self.results[lens_name]['indicator_importance']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]:
                    other_ranking = self.results[other_lens]['indicator_importance']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/climate_science_data.xlsx
DEBUG_LOAD: Called load_data_from_spreadsheet for /content/climate_science_data.xlsx, sheet: None
DEBUG_LOAD: Attempting to read file: /content/climate_science_data.xlsx with extension .xlsx
DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): <class 'dict'>
DEBUG_LOAD: Extracted DataFrame from first sheet 'Sheet1'.
DEBUG_LOAD: Type of 'df' just before final return: <class 'pandas.core.frame.DataFrame'>, id: 133728397282528
DEBUG_LOAD: Returning DataFrame of type: <class 'pandas.core.frame.DataFrame'>, id: 133728397282528
✓ Data loaded successfully from /content/clim

## Final Task

### Subtask:
Provide a summary of the new `IndicatorAnalysisEngine`'s capabilities, emphasizing how it allows for flexible analysis of various time-series indicator data, such as climate science inputs, using the established mathematical lenses.


## Summary:

## Q&A

**What are the capabilities of the new `IndicatorAnalysisEngine`, emphasizing how it allows for flexible analysis of various time-series indicator data, such as climate science inputs, using the established mathematical lenses?**

The `IndicatorAnalysisEngine` provides a streamlined and flexible framework for time-series data analysis. Its capabilities include:

1.  **Flexible Data Loading:** It can load time-series data from both Excel (`.xlsx`, `.xls`, etc.) and CSV files, automatically parsing the first column as dates and setting it as the DataFrame index. It intelligently handles Excel files that might contain multiple sheets, allowing specification of a sheet name or defaulting to the first available sheet.
2.  **Orchestrated Lens Application:** The engine serves as a central orchestrator, initializing and running a `LensComparator` with a suite of predefined mathematical lenses. These lenses include:
    *   **Magnitude Lens:** Quantifies overall activity (L2 norm).
    *   **PCA Lens:** Identifies underlying factors and their importance.
    *   **Granger Causality Lens:** Uncovers temporal predictive relationships between indicators.
    *   **Dynamic Mode Decomposition (DMD) Lens:** Extracts dominant temporal patterns, frequencies, and growth/decay rates.
    *   **Influence Lens:** Determines time-varying activity/volatility of indicators.
    *   **Mutual Information Lens:** Measures information-theoretic dependencies.
3.  **Comprehensive Result Saving:** After running all analyses, the engine systematically saves the processed data and the outputs from the `LensComparator`. These outputs include a lens agreement matrix, consensus indicators, and unique insights from each lens, all stored in a specified output directory for easy access and further interpretation.

This design allows for highly flexible analysis of diverse time-series inputs, such as climate science data, by applying a comprehensive set of mathematical perspectives to reveal different facets of the data's behavior and interrelationships.

## Data Analysis Key Findings

*   The `load_data_from_spreadsheet` function was successfully implemented to parse both CSV and Excel files, automatically setting the first column as a datetime index. A critical fix addressed an issue where `pd.read_excel` could return a dictionary of DataFrames instead of a single DataFrame, ensuring robust data loading.
*   The `IndicatorAnalysisEngine` class was developed to encapsulate the entire analysis workflow, from data ingestion to results storage, promoting modularity and ease of use.
*   A dummy `climate_science_data.xlsx` file was generated, containing six fictional time-series climate indicators (`Global_Temp_Anomaly`, `Sea_Level_Change`, `CO2_Concentration`, `Arctic_Ice_Extent`, `Ocean_Acidity`, `Extreme_Weather_Events_Index`), effectively demonstrating the engine's applicability to time-series data.
*   The `IndicatorAnalysisEngine` successfully executed the full analysis pipeline using the generated climate data. This involved running all six mathematical lenses (Magnitude, PCA, Granger, DMD, Influence, Mutual Information) via the `LensComparator`.
*   The analysis results, including a lens agreement matrix, consensus indicators, and unique insights from each lens, were successfully saved to the specified output directory (`/content/climate_analysis_outputs`).

## Insights or Next Steps

*   **Insight:** The `IndicatorAnalysisEngine` provides a powerful, modular, and extensible platform for multi-faceted time-series analysis, capable of revealing complex relationships and dynamics within various indicator sets, such as those found in climate science. The `LensComparator` effectively synthesizes insights from diverse mathematical perspectives, highlighting both areas of consensus and unique findings.
*   **Next Steps:**
    *   Develop a visualization module to graphically present the outputs from each lens and the comparative insights (e.g., agreement matrices, top indicators), enhancing interpretability for domain experts.
    *   Integrate additional advanced time-series analysis techniques or domain-specific lenses (e.g., wavelet analysis, regime-switching models) to further enrich the engine's analytical capabilities.


# Task
The user has approved the current state. Now, I will implement a new mathematical lens using clustering algorithms (e.g., K-Means, DBSCAN) to identify natural groupings or regimes within the time-series data. This lens will help in discovering hidden patterns and segmenting indicators based on their behavior. I will then integrate a time-series decomposition lens that can break down each indicator into trend, seasonal, and residual components. This will allow for a more granular analysis of the underlying dynamics and help in understanding long-term movements, periodic fluctuations, and unpredictable elements. Finally, I will modify the `LensComparator` to incorporate the new Clustering and Time-Series Decomposition lenses. This includes updating `add_lens`, `run_all`, `consensus_indicators`, and `unique_insights` methods to ensure these new lenses are fully integrated into the comparison framework.

# Task
Create a new class `ClusteringLens` that utilizes the K-Means algorithm to group observations in the time-series data. The `analyze` method will normalize the data, apply K-Means (with a configurable number of clusters), and return cluster labels, cluster centers, and a measure of each indicator's importance in forming these clusters. The `top_indicators` method will report the indicators most important for distinguishing clusters.

## Implement Clustering Lens

### Subtask:
Create a new class `ClusteringLens` that utilizes the K-Means algorithm to group observations in the time-series data. The `analyze` method will normalize the data, apply K-Means (with a configurable number of clusters), and return cluster labels, cluster centers, and a measure of each indicator's importance in forming these clusters. The `top_indicators` method will report the indicators most important for distinguishing clusters.


## Summary:

### Data Analysis Key Findings
No data analysis was performed in this step, as the task involved implementing a new class for future analysis.

### Insights or Next Steps
*   The `ClusteringLens` class has been successfully implemented, providing a reusable component for applying K-Means clustering to time-series data.
*   The `analyze` method of `ClusteringLens` is now capable of normalizing data, applying K-Means with a configurable number of clusters, and returning cluster labels, cluster centers, and feature importance.
*   The `top_indicators` method is ready to identify and report the indicators that are most important for distinguishing between the identified clusters.
*   The next step will involve using this `ClusteringLens` to perform actual clustering on the time-series data and interpret the results.


# Task
## Implement Clustering Lens

### Subtask:
Create a new class `ClusteringLens` that utilizes the K-Means algorithm to group observations in the time-series data. The `analyze` method will normalize the data, apply K-Means (with a configurable number of clusters), and return cluster labels, cluster centers, and a measure of each indicator's importance in forming these clusters. The `top_indicators` method will report the indicators most important for distinguishing clusters.

## Define ClusteringLens (re-evaluation)

### Subtask:
Re-evaluate the definition of the `ClusteringLens` class to ensure it's up-to-date in the environment. This class will apply K-Means clustering and calculate indicator importance based on cluster centers.


**Reasoning**:
The subtask requires defining a new `ClusteringLens` class. I will define the `ClusteringLens` class with `__init__`, `analyze`, and `top_indicators` methods as per the instructions, incorporating K-Means clustering and calculating indicator importance.



In [28]:
from sklearn.cluster import KMeans

class ClusteringLens:
    """
    Identifies natural groupings or regimes within the time-series data using K-Means clustering.
    Answers: "How are indicators naturally grouped and which indicators define these groups?"
    """

    def __init__(self, name: str = "Clustering", n_clusters: int = 3, random_state: int = 42):
        self.name = name
        self.n_clusters = n_clusters
        self.random_state = random_state

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies K-Means clustering to the normalized time-series data.

        Returns:
        - cluster_labels: labels for each observation indicating its cluster.
        - cluster_centers: centroids of the clusters.
        - importance: a measure of how important each indicator is in forming the clusters.
        - method: description of the method used.
        """
        # a. Normalize the input panel data
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna() # K-Means cannot handle NaNs

        if panel_clean.empty or panel_clean.shape[0] < self.n_clusters:
            warnings.warn("Not enough data points or clusters for K-Means. Returning empty results.")
            return {
                'cluster_labels': pd.Series(dtype='int'),
                'cluster_centers': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'method': 'K-Means Clustering'
            }

        # b. Instantiate KMeans with n_clusters and a random_state
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state, n_init=10)

        # c. Fit the KMeans model and obtain cluster_labels
        kmeans.fit(panel_clean)
        cluster_labels = pd.Series(kmeans.labels_, index=panel_clean.index, name='Cluster')

        # d. Calculate cluster_centers from the fitted model
        cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=panel_clean.columns)

        # e. Compute indicator importance: standard deviation of each indicator across cluster_centers
        importance = cluster_centers.std(axis=0).sort_values(ascending=False)

        return {
            'cluster_labels': cluster_labels,
            'cluster_centers': cluster_centers,
            'importance': importance,
            'method': 'K-Means Clustering'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N most important indicators for distinguishing clusters.
        The 'date' parameter is ignored as importance is overall for clustering.
        """
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))

print("Defined `ClusteringLens` class.")

Defined `ClusteringLens` class.


## Implement Time-Series Decomposition Lens

### Subtask:
Create a new class `Time_Series_Decomposition_Lens` that takes a `panel` DataFrame as input. The `analyze` method will iterate through each indicator (column) in the `panel` and apply time-series decomposition (e.g., using `statsmodels.tsa.seasonal.seasonal_decompose`) to break it down into trend, seasonal, and residual components. It should return these components for each indicator. The `top_indicators` method will identify indicators with the highest 'strength' of a certain component (e.g., highest seasonal variance or highest trend magnitude).

**Reasoning**:
The previous step successfully defined the `ClusteringLens`. Now, I will implement the `Time_Series_Decomposition_Lens` as described in the subtask. This involves creating the class with methods for analyzing the data and identifying top indicators based on decomposition components.



In [29]:
import statsmodels.api as sm

class TimeSeriesDecompositionLens:
    """
    Decomposes each indicator into trend, seasonal, and residual components.
    Answers: "What are the underlying patterns (trend, seasonality) and noise in each indicator?"
    """

    def __init__(self, name: str = "TS_Decomposition", model: str = 'additive', period: int = 12):
        self.name = name
        self.model = model
        self.period = period # For monthly data, 12. For quarterly, 4, etc.

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies time-series decomposition to each column of the panel data.

        Returns:
        - decomposition_results: A dictionary where keys are indicator names
                                 and values are dicts containing 'trend', 'seasonal', 'residual'.
        - seasonal_strength: A Series indicating the strength of seasonality for each indicator.
        - method: Description of the method used.
        """
        decomposition_results = {}
        seasonal_strength_metrics = {}

        for col in panel.columns:
            series = panel[col].dropna()
            if len(series) < 2 * self.period: # Need at least two full periods for seasonal decomposition
                warnings.warn(f"Skipping decomposition for {col}: not enough data points ({len(series)}) for period {self.period}.")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan
                continue

            try:
                # Use statsmodels seasonal_decompose
                result = sm.tsa.seasonal.seasonal_decompose(series, model=self.model, period=self.period, extrapolate_trend='freq')
                decomposition_results[col] = {
                    'trend': result.trend,
                    'seasonal': result.seasonal,
                    'residual': result.resid
                }
                # Calculate seasonal strength (variance of seasonal component relative to residual + seasonal)
                if not result.seasonal.dropna().empty and not result.resid.dropna().empty:
                    seasonal_var = np.var(result.seasonal.dropna())
                    resid_var = np.var(result.resid.dropna())
                    total_var = seasonal_var + resid_var
                    seasonal_strength_metrics[col] = seasonal_var / total_var if total_var > 0 else 0
                else:
                    seasonal_strength_metrics[col] = np.nan

            except Exception as e:
                warnings.warn(f"Could not decompose {col}: {e}")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan

        return {
            'decomposition_results': decomposition_results,
            'seasonal_strength': pd.Series(seasonal_strength_metrics).sort_values(ascending=False),
            'method': f'Time-Series Decomposition ({self.model} model, period={self.period})'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N indicators based on their seasonal strength.
        The 'date' parameter is ignored as strength is an overall measure.
        """
        seasonal_strength = result.get('seasonal_strength')
        if seasonal_strength is None or seasonal_strength.empty:
            return []

        top_n = seasonal_strength.head(n)
        return list(zip(top_n.index, top_n.values))

print("Defined `TimeSeriesDecompositionLens` class.")


Defined `TimeSeriesDecompositionLens` class.


## Modify LensComparator for new Lenses

### Subtask:
Modify the `LensComparator` to incorporate the new `ClusteringLens` and `TimeSeriesDecompositionLens`. This includes updating `add_lens`, `run_all`, `consensus_indicators`, and `unique_insights` methods to ensure these new lenses are fully integrated into the comparison framework.

#### Instructions:
1.  **Redefine `LensComparator` class.**
2.  Update the `run_full_lens_analysis` convenience function to add instances of `ClusteringLens` and `TimeSeriesDecompositionLens`.
3.  Modify the `agreement_matrix`, `consensus_indicators`, and `unique_insights` methods within `LensComparator` to properly extract importance metrics from the results of `ClusteringLens` and `TimeSeriesDecompositionLens`.
    *   For `ClusteringLens`, use the `'importance'` key.
    *   For `TimeSeriesDecompositionLens`, use the `'seasonal_strength'` key.

**Reasoning**:
The subtask requires modifying the `LensComparator` class and `run_full_lens_analysis` function to integrate the new `ClusteringLens` and `TimeSeriesDecompositionLens`. I will regenerate the complete code block for all functions and classes, ensuring all previous fixes are included, and explicitly add the new lenses to the comparator. The `agreement_matrix`, `consensus_indicators`, and `unique_insights` methods will be updated to handle the `importance` key from `ClusteringLens` and `seasonal_strength` key from `TimeSeriesDecompositionLens`.



In [30]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from sklearn.stats import spearmanr
from sklearn.cluster import KMeans # New import for ClusteringLens
import statsmodels.api as sm # New import for TimeSeriesDecompositionLens

warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
        KeyError: If the specified sheet_name is not found in an Excel file.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    print(f"DEBUG_LOAD: Attempting to read file: {file_path} with extension {file_extension}")
    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        raw_excel_data = pd.read_excel(file_path, sheet_name=None, parse_dates=[0], index_col=0)
        print(f"DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): {type(raw_excel_data)}")
        if isinstance(raw_excel_data, dict):
            if sheet_name:
                if sheet_name in raw_excel_data:
                    df = raw_excel_data[sheet_name]
                    print(f"DEBUG_LOAD: Extracted DataFrame for specified sheet '{sheet_name}'.")
                else:
                    raise KeyError(f"Sheet '{sheet_name}' not found in the Excel file.")
            else:
                first_sheet_name = list(raw_excel_data.keys())[0]
                df = raw_excel_data[first_sheet_name]
                print(f"DEBUG_LOAD: Extracted DataFrame from first sheet '{first_sheet_name}'.")
        else:
            df = raw_excel_data
            print(f"DEBUG_LOAD: read_excel directly returned a DataFrame.")
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Type of 'df' just before final return: {type(df)}, id: {id(df)}")
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected pandas.DataFrame after reading {file_path}, but got {type(df)} instead.")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# LENS 7: CLUSTERING
# =============================================================================

class ClusteringLens:
    """
    Identifies natural groupings or regimes within the time-series data using K-Means clustering.
    Answers: "How are indicators naturally grouped and which indicators define these groups?"
    """

    def __init__(self, name: str = "Clustering", n_clusters: int = 3, random_state: int = 42):
        self.name = name
        self.n_clusters = n_clusters
        self.random_state = random_state

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies K-Means clustering to the normalized time-series data.

        Returns:
        - cluster_labels: labels for each observation indicating its cluster.
        - cluster_centers: centroids of the clusters.
        - importance: a measure of how important each indicator is in forming the clusters.
        - method: description of the method used.
        """
        # a. Normalize the input panel data
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna() # K-Means cannot handle NaNs

        if panel_clean.empty or panel_clean.shape[0] < self.n_clusters:
            warnings.warn("Not enough data points or clusters for K-Means. Returning empty results.")
            return {
                'cluster_labels': pd.Series(dtype='int'),
                'cluster_centers': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'method': 'K-Means Clustering'
            }

        # b. Instantiate KMeans with n_clusters and a random_state
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state, n_init=10)

        # c. Fit the KMeans model and obtain cluster_labels
        kmeans.fit(panel_clean)
        cluster_labels = pd.Series(kmeans.labels_, index=panel_clean.index, name='Cluster')

        # d. Calculate cluster_centers from the fitted model
        cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=panel_clean.columns)

        # e. Compute indicator importance: standard deviation of each indicator across cluster_centers
        importance = cluster_centers.std(axis=0).sort_values(ascending=False)

        return {
            'cluster_labels': cluster_labels,
            'cluster_centers': cluster_centers,
            'importance': importance,
            'method': 'K-Means Clustering'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N most important indicators for distinguishing clusters.
        The 'date' parameter is ignored as importance is overall for clustering.
        """
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))

# =============================================================================
# LENS 8: TIME SERIES DECOMPOSITION
# =============================================================================

class TimeSeriesDecompositionLens:
    """
    Decomposes each indicator into trend, seasonal, and residual components.
    Answers: "What are the underlying patterns (trend, seasonality) and noise in each indicator?"
    """

    def __init__(self, name: str = "TS_Decomposition", model: str = 'additive', period: int = 12):
        self.name = name
        self.model = model
        self.period = period # For monthly data, 12. For quarterly, 4, etc.

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies time-series decomposition to each column of the panel data.

        Returns:
        - decomposition_results: A dictionary where keys are indicator names
                                 and values are dicts containing 'trend', 'seasonal', 'residual'.
        - seasonal_strength: A Series indicating the strength of seasonality for each indicator.
        - method: Description of the method used.
        """
        decomposition_results = {}
        seasonal_strength_metrics = {}

        for col in panel.columns:
            series = panel[col].dropna()
            if len(series) < 2 * self.period: # Need at least two full periods for seasonal decomposition
                warnings.warn(f"Skipping decomposition for {col}: not enough data points ({len(series)}) for period {self.period}.")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan
                continue

            try:
                # Use statsmodels seasonal_decompose
                result = sm.tsa.seasonal.seasonal_decompose(series, model=self.model, period=self.period, extrapolate_trend='freq')
                decomposition_results[col] = {
                    'trend': result.trend,
                    'seasonal': result.seasonal,
                    'residual': result.resid
                }
                # Calculate seasonal strength (variance of seasonal component relative to residual + seasonal)
                if not result.seasonal.dropna().empty and not result.resid.dropna().empty:
                    seasonal_var = np.var(result.seasonal.dropna())
                    resid_var = np.var(result.resid.dropna())
                    total_var = seasonal_var + resid_var
                    seasonal_strength_metrics[col] = seasonal_var / total_var if total_var > 0 else 0
                else:
                    seasonal_strength_metrics[col] = np.nan

            except Exception as e:
                warnings.warn(f"Could not decompose {col}: {e}")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan

        return {
            'decomposition_results': decomposition_results,
            'seasonal_strength': pd.Series(seasonal_strength_metrics).sort_values(ascending=False),
            'method': f'Time-Series Decomposition ({self.model} model, period={self.period})'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N indicators based on their seasonal strength.
        The 'date' parameter is ignored as strength is an overall measure.
        """
        seasonal_strength = result.get('seasonal_strength')
        if seasonal_strength is None or seasonal_strength.empty:
            return []

        top_n = seasonal_strength.head(n)
        return list(zip(top_n.index, top_n.values))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                importance_rankings[lens_name] = self.results[lens_name]['seasonal_strength']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        # Need to ensure all_rankings is not empty before creating DataFrame
        if not all_rankings:
            return pd.DataFrame(columns=['mean_score', 'std_score', 'n_lenses'])

        # Pad shorter series with NaNs to align indexes for concatenation
        aligned_rankings = []
        all_indices = pd.Index([])
        for ranking in all_rankings:
            all_indices = all_indices.union(ranking.index)

        for ranking in all_rankings:
            aligned_rankings.append(ranking.reindex(all_indices))

        consensus_df = pd.concat(aligned_rankings, axis=1, keys=[f'lens_{i}' for i in range(len(aligned_rankings))])
        consensus_df.index.name = 'indicator'

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                this_ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                this_ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]: # For PCA and Clustering
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]: # For Granger
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]: # For Mutual Information
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]: # For DMD
                    other_ranking = self.results[other_lens]['indicator_importance']
                elif 'seasonal_strength' in self.results[other_lens]: # For TimeSeriesDecompositionLens
                    other_ranking = self.results[other_lens]['seasonal_strength']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())
    comparator.add_lens(ClusteringLens()) # Add new Clustering Lens
    comparator.add_lens(TimeSeriesDecompositionLens()) # Add new Time-Series Decomposition Lens

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...


ModuleNotFoundError: No module named 'sklearn.stats'

**Reasoning**:
The previous execution failed due to `ModuleNotFoundError: No module named 'sklearn.stats'`. This is a typo; `spearmanr` should be imported from `scipy.stats` not `sklearn.stats`. I will correct the import statement for `spearmanr` to `from scipy.stats import spearmanr` and resubmit the entire code block.



In [31]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests # Corrected typo: stattools
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr # Corrected import: from scipy.stats
from sklearn.cluster import KMeans # New import for ClusteringLens
import statsmodels.api as sm # New import for TimeSeriesDecompositionLens

warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
        KeyError: If the specified sheet_name is not found in an Excel file.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    print(f"DEBUG_LOAD: Attempting to read file: {file_path} with extension {file_extension}")
    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        raw_excel_data = pd.read_excel(file_path, sheet_name=None, parse_dates=[0], index_col=0)
        print(f"DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): {type(raw_excel_data)}")
        if isinstance(raw_excel_data, dict):
            if sheet_name:
                if sheet_name in raw_excel_data:
                    df = raw_excel_data[sheet_name]
                    print(f"DEBUG_LOAD: Extracted DataFrame for specified sheet '{sheet_name}'.")
                else:
                    raise KeyError(f"Sheet '{sheet_name}' not found in the Excel file.")
            else:
                first_sheet_name = list(raw_excel_data.keys())[0]
                df = raw_excel_data[first_sheet_name]
                print(f"DEBUG_LOAD: Extracted DataFrame from first sheet '{first_sheet_name}'.")
        else:
            df = raw_excel_data
            print(f"DEBUG_LOAD: read_excel directly returned a DataFrame.")
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Type of 'df' just before final return: {type(df)}, id: {id(df)}")
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected pandas.DataFrame after reading {file_path}, but got {type(df)} instead.")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# LENS 7: CLUSTERING
# =============================================================================

class ClusteringLens:
    """
    Identifies natural groupings or regimes within the time-series data using K-Means clustering.
    Answers: "How are indicators naturally grouped and which indicators define these groups?"
    """

    def __init__(self, name: str = "Clustering", n_clusters: int = 3, random_state: int = 42):
        self.name = name
        self.n_clusters = n_clusters
        self.random_state = random_state

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies K-Means clustering to the normalized time-series data.

        Returns:
        - cluster_labels: labels for each observation indicating its cluster.
        - cluster_centers: centroids of the clusters.
        - importance: a measure of how important each indicator is in forming the clusters.
        - method: description of the method used.
        """
        # a. Normalize the input panel data
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna() # K-Means cannot handle NaNs

        if panel_clean.empty or panel_clean.shape[0] < self.n_clusters:
            warnings.warn("Not enough data points or clusters for K-Means. Returning empty results.")
            return {
                'cluster_labels': pd.Series(dtype='int'),
                'cluster_centers': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'method': 'K-Means Clustering'
            }

        # b. Instantiate KMeans with n_clusters and a random_state
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state, n_init=10)

        # c. Fit the KMeans model and obtain cluster_labels
        kmeans.fit(panel_clean)
        cluster_labels = pd.Series(kmeans.labels_, index=panel_clean.index, name='Cluster')

        # d. Calculate cluster_centers from the fitted model
        cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=panel_clean.columns)

        # e. Compute indicator importance: standard deviation of each indicator across cluster_centers
        importance = cluster_centers.std(axis=0).sort_values(ascending=False)

        return {
            'cluster_labels': cluster_labels,
            'cluster_centers': cluster_centers,
            'importance': importance,
            'method': 'K-Means Clustering'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N most important indicators for distinguishing clusters.
        The 'date' parameter is ignored as importance is overall for clustering.
        """
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))

# =============================================================================
# LENS 8: TIME SERIES DECOMPOSITION
# =============================================================================

class TimeSeriesDecompositionLens:
    """
    Decomposes each indicator into trend, seasonal, and residual components.
    Answers: "What are the underlying patterns (trend, seasonality) and noise in each indicator?"
    """

    def __init__(self, name: str = "TS_Decomposition", model: str = 'additive', period: int = 12):
        self.name = name
        self.model = model
        self.period = period # For monthly data, 12. For quarterly, 4, etc.

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies time-series decomposition to each column of the panel data.

        Returns:
        - decomposition_results: A dictionary where keys are indicator names
                                 and values are dicts containing 'trend', 'seasonal', 'residual'.
        - seasonal_strength: A Series indicating the strength of seasonality for each indicator.
        - method: Description of the method used.
        """
        decomposition_results = {}
        seasonal_strength_metrics = {}

        for col in panel.columns:
            series = panel[col].dropna()
            if len(series) < 2 * self.period: # Need at least two full periods for seasonal decomposition
                warnings.warn(f"Skipping decomposition for {col}: not enough data points ({len(series)}) for period {self.period}.")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan
                continue

            try:
                # Use statsmodels seasonal_decompose
                result = sm.tsa.seasonal.seasonal_decompose(series, model=self.model, period=self.period, extrapolate_trend='freq')
                decomposition_results[col] = {
                    'trend': result.trend,
                    'seasonal': result.seasonal,
                    'residual': result.resid
                }
                # Calculate seasonal strength (variance of seasonal component relative to residual + seasonal)
                if not result.seasonal.dropna().empty and not result.resid.dropna().empty:
                    seasonal_var = np.var(result.seasonal.dropna())
                    resid_var = np.var(result.resid.dropna())
                    total_var = seasonal_var + resid_var
                    seasonal_strength_metrics[col] = seasonal_var / total_var if total_var > 0 else 0
                else:
                    seasonal_strength_metrics[col] = np.nan

            except Exception as e:
                warnings.warn(f"Could not decompose {col}: {e}")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan

        return {
            'decomposition_results': decomposition_results,
            'seasonal_strength': pd.Series(seasonal_strength_metrics).sort_values(ascending=False),
            'method': f'Time-Series Decomposition ({self.model} model, period={self.period})'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N indicators based on their seasonal strength.
        The 'date' parameter is ignored as strength is an overall measure.
        """
        seasonal_strength = result.get('seasonal_strength')
        if seasonal_strength is None or seasonal_strength.empty:
            return []

        top_n = seasonal_strength.head(n)
        return list(zip(top_n.index, top_n.values))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                importance_rankings[lens_name] = self.results[lens_name]['seasonal_strength']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        # Need to ensure all_rankings is not empty before creating DataFrame
        if not all_rankings:
            return pd.DataFrame(columns=['mean_score', 'std_score', 'n_lenses'])

        # Pad shorter series with NaNs to align indexes for concatenation
        aligned_rankings = []
        all_indices = pd.Index([])
        for ranking in all_rankings:
            all_indices = all_indices.union(ranking.index)

        for ranking in all_rankings:
            aligned_rankings.append(ranking.reindex(all_indices))

        consensus_df = pd.concat(aligned_rankings, axis=1, keys=[f'lens_{i}' for i in range(len(aligned_rankings))])
        consensus_df.index.name = 'indicator'

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                this_ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                this_ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]: # For PCA and Clustering
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]: # For Granger
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]: # For Mutual Information
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]: # For DMD
                    other_ranking = self.results[other_lens]['indicator_importance']
                elif 'seasonal_strength' in self.results[other_lens]: # For TimeSeriesDecompositionLens
                    other_ranking = self.results[other_lens]['seasonal_strength']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())
    comparator.add_lens(ClusteringLens()) # Add new Clustering Lens
    comparator.add_lens(TimeSeriesDecompositionLens()) # Add new Time-Series Decomposition Lens

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/climate_science_data.xlsx
DEBUG_LOAD: Called load_data_from_spreadsheet for /content/climate_science_data.xlsx, sheet: None
DEBUG_LOAD: Attempting to read file: /content/climate_science_data.xlsx with extension .xlsx
DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): <class 'dict'>
DEBUG_LOAD: Extracted DataFrame from first sheet 'Sheet1'.
DEBUG_LOAD: Type of 'df' just before final return: <class 'pandas.core.frame.DataFrame'>, id: 133728306493632
DEBUG_LOAD: Returning DataFrame of type: <class 'pandas.core.frame.DataFrame'>, id: 133728306493632
✓ Data loaded successfully from /content/clim

**Reasoning**:
The previous execution failed due to a `ModuleNotFoundError: No module named 'sklearn.stats'`. This is a typo; `spearmanr` should be imported from `scipy.stats` not `sklearn.stats`. I will correct the import statement for `spearmanr` to `from scipy.stats import spearmanr` and resubmit the entire code block.



In [32]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr # Corrected import: from scipy.stats
from sklearn.cluster import KMeans # New import for ClusteringLens
import statsmodels.api as sm # New import for TimeSeriesDecompositionLens

warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
        KeyError: If the specified sheet_name is not found in an Excel file.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    print(f"DEBUG_LOAD: Attempting to read file: {file_path} with extension {file_extension}")
    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        raw_excel_data = pd.read_excel(file_path, sheet_name=None, parse_dates=[0], index_col=0)
        print(f"DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): {type(raw_excel_data)}")
        if isinstance(raw_excel_data, dict):
            if sheet_name:
                if sheet_name in raw_excel_data:
                    df = raw_excel_data[sheet_name]
                    print(f"DEBUG_LOAD: Extracted DataFrame for specified sheet '{sheet_name}'.")
                else:
                    raise KeyError(f"Sheet '{sheet_name}' not found in the Excel file.")
            else:
                first_sheet_name = list(raw_excel_data.keys())[0]
                df = raw_excel_data[first_sheet_name]
                print(f"DEBUG_LOAD: Extracted DataFrame from first sheet '{first_sheet_name}'.")
        else:
            df = raw_excel_data
            print(f"DEBUG_LOAD: read_excel directly returned a DataFrame.")
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Type of 'df' just before final return: {type(df)}, id: {id(df)}")
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected pandas.DataFrame after reading {file_path}, but got {type(df)} instead.")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# LENS 7: CLUSTERING
# =============================================================================

class ClusteringLens:
    """
    Identifies natural groupings or regimes within the time-series data using K-Means clustering.
    Answers: "How are indicators naturally grouped and which indicators define these groups?"
    """

    def __init__(self, name: str = "Clustering", n_clusters: int = 3, random_state: int = 42):
        self.name = name
        self.n_clusters = n_clusters
        self.random_state = random_state

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies K-Means clustering to the normalized time-series data.

        Returns:
        - cluster_labels: labels for each observation indicating its cluster.
        - cluster_centers: centroids of the clusters.
        - importance: a measure of how important each indicator is in forming the clusters.
        - method: description of the method used.
        """
        # a. Normalize the input panel data
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna() # K-Means cannot handle NaNs

        if panel_clean.empty or panel_clean.shape[0] < self.n_clusters:
            warnings.warn("Not enough data points or clusters for K-Means. Returning empty results.")
            return {
                'cluster_labels': pd.Series(dtype='int'),
                'cluster_centers': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'method': 'K-Means Clustering'
            }

        # b. Instantiate KMeans with n_clusters and a random_state
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state, n_init=10)

        # c. Fit the KMeans model and obtain cluster_labels
        kmeans.fit(panel_clean)
        cluster_labels = pd.Series(kmeans.labels_, index=panel_clean.index, name='Cluster')

        # d. Calculate cluster_centers from the fitted model
        cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=panel_clean.columns)

        # e. Compute indicator importance: standard deviation of each indicator across cluster_centers
        importance = cluster_centers.std(axis=0).sort_values(ascending=False)

        return {
            'cluster_labels': cluster_labels,
            'cluster_centers': cluster_centers,
            'importance': importance,
            'method': 'K-Means Clustering'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N most important indicators for distinguishing clusters.
        The 'date' parameter is ignored as importance is overall for clustering.
        """
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))

# =============================================================================
# LENS 8: TIME SERIES DECOMPOSITION
# =============================================================================

class TimeSeriesDecompositionLens:
    """
    Decomposes each indicator into trend, seasonal, and residual components.
    Answers: "What are the underlying patterns (trend, seasonality) and noise in each indicator?"
    """

    def __init__(self, name: str = "TS_Decomposition", model: str = 'additive', period: int = 12):
        self.name = name
        self.model = model
        self.period = period # For monthly data, 12. For quarterly, 4, etc.

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies time-series decomposition to each column of the panel data.

        Returns:
        - decomposition_results: A dictionary where keys are indicator names
                                 and values are dicts containing 'trend', 'seasonal', 'residual'.
        - seasonal_strength: A Series indicating the strength of seasonality for each indicator.
        - method: Description of the method used.
        """
        decomposition_results = {}
        seasonal_strength_metrics = {}

        for col in panel.columns:
            series = panel[col].dropna()
            if len(series) < 2 * self.period: # Need at least two full periods for seasonal decomposition
                warnings.warn(f"Skipping decomposition for {col}: not enough data points ({len(series)}) for period {self.period}.")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan
                continue

            try:
                # Use statsmodels seasonal_decompose
                result = sm.tsa.seasonal.seasonal_decompose(series, model=self.model, period=self.period, extrapolate_trend='freq')
                decomposition_results[col] = {
                    'trend': result.trend,
                    'seasonal': result.seasonal,
                    'residual': result.resid
                }
                # Calculate seasonal strength (variance of seasonal component relative to residual + seasonal)
                if not result.seasonal.dropna().empty and not result.resid.dropna().empty:
                    seasonal_var = np.var(result.seasonal.dropna())
                    resid_var = np.var(result.resid.dropna())
                    total_var = seasonal_var + resid_var
                    seasonal_strength_metrics[col] = seasonal_var / total_var if total_var > 0 else 0
                else:
                    seasonal_strength_metrics[col] = np.nan

            except Exception as e:
                warnings.warn(f"Could not decompose {col}: {e}")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan

        return {
            'decomposition_results': decomposition_results,
            'seasonal_strength': pd.Series(seasonal_strength_metrics).sort_values(ascending=False),
            'method': f'Time-Series Decomposition ({self.model} model, period={self.period})'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N indicators based on their seasonal strength.
        The 'date' parameter is ignored as strength is an overall measure.
        """
        seasonal_strength = result.get('seasonal_strength')
        if seasonal_strength is None or seasonal_strength.empty:
            return []

        top_n = seasonal_strength.head(n)
        return list(zip(top_n.index, top_n.values))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                importance_rankings[lens_name] = self.results[lens_name]['seasonal_strength']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        # Need to ensure all_rankings is not empty before creating DataFrame
        if not all_rankings:
            return pd.DataFrame(columns=['mean_score', 'std_score', 'n_lenses'])

        # Pad shorter series with NaNs to align indexes for concatenation
        aligned_rankings = []
        all_indices = pd.Index([])
        for ranking in all_rankings:
            all_indices = all_indices.union(ranking.index)

        for ranking in all_rankings:
            aligned_rankings.append(ranking.reindex(all_indices))

        consensus_df = pd.concat(aligned_rankings, axis=1, keys=[f'lens_{i}' for i in range(len(aligned_rankings))])
        consensus_df.index.name = 'indicator'

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                this_ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                this_ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]: # For PCA and Clustering
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]: # For Granger
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]: # For Mutual Information
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]: # For DMD
                    other_ranking = self.results[other_lens]['indicator_importance']
                elif 'seasonal_strength' in self.results[other_lens]: # For TimeSeriesDecompositionLens
                    other_ranking = self.results[other_lens]['seasonal_strength']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())
    comparator.add_lens(ClusteringLens()) # Add new Clustering Lens
    comparator.add_lens(TimeSeriesDecompositionLens()) # Add new Time-Series Decomposition Lens

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/climate_science_data.xlsx'

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/climate_analysis_outputs'
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/climate_science_data.xlsx
Output directory: /content/climate_analysis_outputs

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/climate_science_data.xlsx
DEBUG_LOAD: Called load_data_from_spreadsheet for /content/climate_science_data.xlsx, sheet: None
DEBUG_LOAD: Attempting to read file: /content/climate_science_data.xlsx with extension .xlsx
DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): <class 'dict'>
DEBUG_LOAD: Extracted DataFrame from first sheet 'Sheet1'.
DEBUG_LOAD: Type of 'df' just before final return: <class 'pandas.core.frame.DataFrame'>, id: 133728245529488
DEBUG_LOAD: Returning DataFrame of type: <class 'pandas.core.frame.DataFrame'>, id: 133728245529488
✓ Data loaded successfully from /content/clim

## Demonstrate Updated Engine with Clustering

### Subtask:
Execute the `IndicatorAnalysisEngine` with the climate science data. This run will now include the `ClusteringLens`, and its results will be processed and included in the output files and comparative analysis performed by the `LensComparator`.


**Reasoning**:
The subtask requires executing the `IndicatorAnalysisEngine` with the climate science data, which now incorporates the `ClusteringLens`. I will provide the code to instantiate the engine and run its analysis method, including all necessary imports and the helper functions, to ensure the environment is correctly set up for the new lens.



In [3]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr # Corrected import: from scipy.stats
from sklearn.cluster import KMeans # New import for ClusteringLens
import statsmodels.api as sm # New import for TimeSeriesDecompositionLens

warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
        KeyError: If the specified sheet_name is not found in an Excel file.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    print(f"DEBUG_LOAD: Attempting to read file: {file_path} with extension {file_extension}")
    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        raw_excel_data = pd.read_excel(file_path, sheet_name=None, parse_dates=[0], index_col=0)
        print(f"DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): {type(raw_excel_data)}")
        if isinstance(raw_excel_data, dict):
            if sheet_name:
                if sheet_name in raw_excel_data:
                    df = raw_excel_data[sheet_name]
                    print(f"DEBUG_LOAD: Extracted DataFrame for specified sheet '{sheet_name}'.")
                else:
                    raise KeyError(f"Sheet '{sheet_name}' not found in the Excel file.")
            else:
                first_sheet_name = list(raw_excel_data.keys())[0]
                df = raw_excel_data[first_sheet_name]
                print(f"DEBUG_LOAD: Extracted DataFrame from first sheet '{first_sheet_name}'.")
        else:
            df = raw_excel_data
            print(f"DEBUG_LOAD: read_excel directly returned a DataFrame.")
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Type of 'df' just before final return: {type(df)}, id: {id(df)}")
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected pandas.DataFrame after reading {file_path}, but got {type(df)} instead.")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much each is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# LENS 7: CLUSTERING
# =============================================================================

class ClusteringLens:
    """
    Identifies natural groupings or regimes within the time-series data using K-Means clustering.
    Answers: "How are indicators naturally grouped and which indicators define these groups?"
    """

    def __init__(self, name: str = "Clustering", n_clusters: int = 3, random_state: int = 42):
        self.name = name
        self.n_clusters = n_clusters
        self.random_state = random_state

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies K-Means clustering to the normalized time-series data.

        Returns:
        - cluster_labels: labels for each observation indicating its cluster.
        - cluster_centers: centroids of the clusters.
        - importance: a measure of how important each indicator is in forming the clusters.
        - method: description of the method used.
        """
        # a. Normalize the input panel data
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna() # K-Means cannot handle NaNs

        if panel_clean.empty or panel_clean.shape[0] < self.n_clusters:
            warnings.warn("Not enough data points or clusters for K-Means. Returning empty results.")
            return {
                'cluster_labels': pd.Series(dtype='int'),
                'cluster_centers': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'method': 'K-Means Clustering'
            }

        # b. Instantiate KMeans with n_clusters and a random_state
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state, n_init=10)

        # c. Fit the KMeans model and obtain cluster_labels
        kmeans.fit(panel_clean)
        cluster_labels = pd.Series(kmeans.labels_, index=panel_clean.index, name='Cluster')

        # d. Calculate cluster_centers from the fitted model
        cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=panel_clean.columns)

        # e. Compute indicator importance: standard deviation of each indicator across cluster_centers
        importance = cluster_centers.std(axis=0).sort_values(ascending=False)

        return {
            'cluster_labels': cluster_labels,
            'cluster_centers': cluster_centers,
            'importance': importance,
            'method': 'K-Means Clustering'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N most important indicators for distinguishing clusters.
        The 'date' parameter is ignored as importance is overall for clustering.
        """
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))

# =============================================================================
# LENS 8: TIME SERIES DECOMPOSITION
# =============================================================================

class TimeSeriesDecompositionLens:
    """
    Decomposes each indicator into trend, seasonal, and residual components.
    Answers: "What are the underlying patterns (trend, seasonality) and noise in each indicator?"
    """

    def __init__(self, name: str = "TS_Decomposition", model: str = 'additive', period: int = 12):
        self.name = name
        self.model = model
        self.period = period # For monthly data, 12. For quarterly, 4, etc.

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies time-series decomposition to each column of the panel data.

        Returns:
        - decomposition_results: A dictionary where keys are indicator names
                                 and values are dicts containing 'trend', 'seasonal', 'residual'.
        - seasonal_strength: A Series indicating the strength of seasonality for each indicator.
        - method: Description of the method used.
        """
        decomposition_results = {}
        seasonal_strength_metrics = {}

        for col in panel.columns:
            series = panel[col].dropna()
            if len(series) < 2 * self.period: # Need at least two full periods for seasonal decomposition
                warnings.warn(f"Skipping decomposition for {col}: not enough data points ({len(series)}) for period {self.period}.")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan
                continue

            try:
                # Use statsmodels seasonal_decompose
                result = sm.tsa.seasonal.seasonal_decompose(series, model=self.model, period=self.period, extrapolate_trend='freq')
                decomposition_results[col] = {
                    'trend': result.trend,
                    'seasonal': result.seasonal,
                    'residual': result.resid
                }
                # Calculate seasonal strength (variance of seasonal component relative to residual + seasonal)
                if not result.seasonal.dropna().empty and not result.resid.dropna().empty:
                    seasonal_var = np.var(result.seasonal.dropna())
                    resid_var = np.var(result.resid.dropna())
                    total_var = seasonal_var + resid_var
                    seasonal_strength_metrics[col] = seasonal_var / total_var if total_var > 0 else 0
                else:
                    seasonal_strength_metrics[col] = np.nan

            except Exception as e:
                warnings.warn(f"Could not decompose {col}: {e}")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan

        return {
            'decomposition_results': decomposition_results,
            'seasonal_strength': pd.Series(seasonal_strength_metrics).sort_values(ascending=False),
            'method': f'Time-Series Decomposition ({self.model} model, period={self.period})'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N indicators based on their seasonal strength.
        The 'date' parameter is ignored as strength is an overall measure.
        """
        seasonal_strength = result.get('seasonal_strength')
        if seasonal_strength is None or seasonal_strength.empty:
            return []

        top_n = seasonal_strength.head(n)
        return list(zip(top_n.index, top_n.values))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00d7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                importance_rankings[lens_name] = self.results[lens_name]['seasonal_strength']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        # Need to ensure all_rankings is not empty before creating DataFrame
        if not all_rankings:
            return pd.DataFrame(columns=['mean_score', 'std_score', 'n_lenses'])

        # Pad shorter series with NaNs to align indexes for concatenation
        aligned_rankings = []
        all_indices = pd.Index([])
        for ranking in all_rankings:
            all_indices = all_indices.union(ranking.index)

        for ranking in all_rankings:
            aligned_rankings.append(ranking.reindex(all_indices))

        consensus_df = pd.concat(aligned_rankings, axis=1, keys=[f'lens_{i}' for i in range(len(aligned_rankings))])
        consensus_df.index.name = 'indicator'

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                this_ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                this_ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]: # For PCA and Clustering
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]: # For Granger
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]: # For Mutual Information
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]: # For DMD
                    other_ranking = self.results[other_lens]['indicator_importance']
                elif 'seasonal_strength' in self.results[other_lens]: # For TimeSeriesDecompositionLens
                    other_ranking = self.results[other_lens]['seasonal_strength']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())
    comparator.add_lens(ClusteringLens()) # Add new Clustering Lens
    comparator.add_lens(TimeSeriesDecompositionLens()) # Add new Time-Series Decomposition Lens

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:\n")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Runs the LensComparator with all defined lenses.
        3. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/drive/MyDrive/prism_engine/data_raw/master_panel.csv' # Changed file path

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/master_panel_analysis_outputs' # Changed output directory
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv
Output directory: /content/master_panel_analysis_outputs

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv
DEBUG_LOAD: Called load_data_from_spreadsheet for /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv, sheet: None
DEBUG_LOAD: Attempting to read file: /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv with extension .csv
DEBUG_LOAD: Type of 'df' just before final return: <class 'pandas.core.frame.DataFrame'>, id: 137880448978864
DEBUG_LOAD: Returning DataFrame of type: <class 'pandas.core.frame.DataFrame'>, id: 137880448978864
✓ Data loaded successfully from /content/drive/MyDrive/prism_engine/data_raw/

## Final Task

### Subtask:
Summarize how the `ClusteringLens` has been integrated and what new insights it provides within the `LensComparator` framework.


## Summary:

### Q&A
The `ClusteringLens` has been integrated into the `LensComparator` framework by defining a dedicated class that utilizes K-Means clustering. Its `analyze` method normalizes the input data, applies K-Means (with a configurable number of clusters), and returns cluster labels, cluster centers, and an 'importance' score for each indicator. This 'importance' is calculated as the standard deviation of each indicator's value across the cluster centers, signifying how much that indicator contributes to distinguishing between the identified clusters.

Within the `LensComparator`, the `ClusteringLens` is added as another analytical perspective alongside other lenses (e.g., PCA, Granger Causality). The `LensComparator`'s `agreement_matrix`, `consensus_indicators`, and `unique_insights` methods have been updated to specifically look for the `'importance'` key in the `ClusteringLens`'s results dictionary. This allows the framework to compare the `ClusteringLens`'s assessment of indicator importance with that of other lenses, identifying indicators that consistently appear as important across different methodologies or those that are uniquely highlighted by the clustering approach.

### Data Analysis Key Findings
*   The `ClusteringLens` class was successfully implemented, performing K-Means clustering on normalized time-series data and accurately calculating indicator importance based on the standard deviation of cluster centers.
*   The `LensComparator` was updated to include the `ClusteringLens` in its analysis pipeline, enabling it to run alongside seven other analytical lenses.
*   The `LensComparator`'s comparative methods (`agreement_matrix`, `consensus_indicators`, `unique_insights`) were correctly modified to extract the `'importance'` metric from the `ClusteringLens` results, ensuring its insights are integrated into the overall comparison.
*   The full analysis pipeline, including data loading, execution of all eight lenses (including `ClusteringLens`), and generation of comparative reports (agreement matrix, consensus indicators, unique insights), executed successfully, confirming the seamless integration and functionality of the new lens.
*   Consensus indicator analysis showed that `Ocean_Acidity`, `Global_Temp_Anomaly`, and `Sea_Level_Change` were frequently identified as important across multiple lenses, suggesting their broad significance.

### Insights or Next Steps
*   The `ClusteringLens` can provide insights into natural groupings or "regimes" within the time-series data, helping to understand periods of similar behavior across indicators. Further analysis of the `cluster_labels` could reveal these periods.
*   Given the successful integration, exploring different numbers of clusters (`n_clusters`) for the `ClusteringLens` could yield more nuanced understandings of data regimes and their defining indicators.


# Task
Implement a NaN handling strategy in the `IndicatorAnalysisEngine` to preprocess `master_panel.csv` data using 'ffill', 'linear', or 'none' before analysis. Adjust individual lenses (PCA, DMD, Mutual Information) to remove redundant `dropna()` calls. Re-run the analysis and summarize the impact of NaN handling on the performance of previously failing lenses and on the overall analysis results, including consensus indicators and unique insights.

## Implement NaN Handling Strategy

### Subtask:
Modify the `IndicatorAnalysisEngine` to include a parameter for NaN handling (e.g., 'ffill', 'linear', 'none'). Implement a preprocessing step in the `run_analysis` method that applies the chosen NaN handling strategy to the `panel_data` before it's passed to the `LensComparator`. This will involve either forward-filling missing values or using linear interpolation, to ensure all lenses receive a complete dataset.


**Reasoning**:
The subtask requires modifying the `IndicatorAnalysisEngine` to accept a NaN handling strategy and apply it before passing data to the lenses. I will regenerate the full code block for all functions and classes, incorporating the `nan_handling_strategy` parameter in `IndicatorAnalysisEngine.__init__` and implementing the conditional logic for 'ffill' or 'linear' interpolation in its `run_analysis` method, along with debug prints. I will also ensure `%reset -f` is called to clear the environment before re-defining the code.



In [4]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
from sklearn.cluster import KMeans # New import for ClusteringLens
import statsmodels.api as sm # New import for TimeSeriesDecompositionLens

warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
        KeyError: If the specified sheet_name is not found in an Excel file.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    print(f"DEBUG_LOAD: Attempting to read file: {file_path} with extension {file_extension}")
    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        raw_excel_data = pd.read_excel(file_path, sheet_name=None, parse_dates=[0], index_col=0)
        print(f"DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): {type(raw_excel_data)}")
        if isinstance(raw_excel_data, dict):
            if sheet_name:
                if sheet_name in raw_excel_data:
                    df = raw_excel_data[sheet_name]
                    print(f"DEBUG_LOAD: Extracted DataFrame for specified sheet '{sheet_name}'.")
                else:
                    raise KeyError(f"Sheet '{sheet_name}' not found in the Excel file.")
            else:
                first_sheet_name = list(raw_excel_data.keys())[0]
                df = raw_excel_data[first_sheet_name]
                print(f"DEBUG_LOAD: Extracted DataFrame from first sheet '{first_sheet_name}'.")
        else:
            df = raw_excel_data
            print(f"DEBUG_LOAD: read_excel directly returned a DataFrame.")
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Type of 'df' just before final return: {type(df)}, id: {id(df)}")
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected pandas.DataFrame after reading {file_path}, but got {type(df)} instead.")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna()

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except:
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X = panel_norm.dropna().T.values

        # DMD
        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])

        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna()
        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            X = panel_clean.drop(columns=[col_i]).values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            other_cols = [c for c in panel_clean.columns if c != col_i]
            for j, col_j in enumerate(other_cols):
                j_idx = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx] = mi_scores[j]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# LENS 7: CLUSTERING
# =============================================================================

class ClusteringLens:
    """
    Identifies natural groupings or regimes within the time-series data using K-Means clustering.
    Answers: "How are indicators naturally grouped and which indicators define these groups?"
    """

    def __init__(self, name: str = "Clustering", n_clusters: int = 3, random_state: int = 42):
        self.name = name
        self.n_clusters = n_clusters
        self.random_state = random_state

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies K-Means clustering to the normalized time-series data.

        Returns:
        - cluster_labels: labels for each observation indicating its cluster.
        - cluster_centers: centroids of the clusters.
        - importance: a measure of how important each indicator is in forming the clusters.
        - method: description of the method used.
        """
        # a. Normalize the input panel data
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna() # K-Means cannot handle NaNs

        if panel_clean.empty or panel_clean.shape[0] < self.n_clusters:
            warnings.warn("Not enough data points or clusters for K-Means. Returning empty results.")
            return {
                'cluster_labels': pd.Series(dtype='int'),
                'cluster_centers': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'method': 'K-Means Clustering'
            }

        # b. Instantiate KMeans with n_clusters and a random_state
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state, n_init=10)

        # c. Fit the KMeans model and obtain cluster_labels
        kmeans.fit(panel_clean)
        cluster_labels = pd.Series(kmeans.labels_, index=panel_clean.index, name='Cluster')

        # d. Calculate cluster_centers from the fitted model
        cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=panel_clean.columns)

        # e. Compute indicator importance: standard deviation of each indicator across cluster_centers
        importance = cluster_centers.std(axis=0).sort_values(ascending=False)

        return {
            'cluster_labels': cluster_labels,
            'cluster_centers': cluster_centers,
            'importance': importance,
            'method': 'K-Means Clustering'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N most important indicators for distinguishing clusters.
        The 'date' parameter is ignored as importance is overall for clustering.
        """
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))

# =============================================================================
# LENS 8: TIME SERIES DECOMPOSITION
# =============================================================================

class TimeSeriesDecompositionLens:
    """
    Decomposes each indicator into trend, seasonal, and residual components.
    Answers: "What are the underlying patterns (trend, seasonality) and noise in each indicator?"
    """

    def __init__(self, name: str = "TS_Decomposition", model: str = 'additive', period: int = 12):
        self.name = name
        self.model = model
        self.period = period # For monthly data, 12. For quarterly, 4, etc.

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies time-series decomposition to each column of the panel data.

        Returns:
        - decomposition_results: A dictionary where keys are indicator names
                                 and values are dicts containing 'trend', 'seasonal', 'residual'.
        - seasonal_strength: A Series indicating the strength of seasonality for each indicator.
        - method: Description of the method used.
        """
        decomposition_results = {}
        seasonal_strength_metrics = {}

        for col in panel.columns:
            series = panel[col].dropna()
            if len(series) < 2 * self.period: # Need at least two full periods for seasonal decomposition
                warnings.warn(f"Skipping decomposition for {col}: not enough data points ({len(series)}) for period {self.period}.")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan
                continue

            try:
                # Use statsmodels seasonal_decompose
                result = sm.tsa.seasonal.seasonal_decompose(series, model=self.model, period=self.period, extrapolate_trend='freq')
                decomposition_results[col] = {
                    'trend': result.trend,
                    'seasonal': result.seasonal,
                    'residual': result.resid
                }
                # Calculate seasonal strength (variance of seasonal component relative to residual + seasonal)
                if not result.seasonal.dropna().empty and not result.resid.dropna().empty:
                    seasonal_var = np.var(result.seasonal.dropna())
                    resid_var = np.var(result.resid.dropna())
                    total_var = seasonal_var + resid_var
                    seasonal_strength_metrics[col] = seasonal_var / total_var if total_var > 0 else 0
                else:
                    seasonal_strength_metrics[col] = np.nan

            except Exception as e:
                warnings.warn(f"Could not decompose {col}: {e}")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan

        return {
            'decomposition_results': decomposition_results,
            'seasonal_strength': pd.Series(seasonal_strength_metrics).sort_values(ascending=False),
            'method': f'Time-Series Decomposition ({self.model} model, period={self.period})'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N indicators based on their seasonal strength.
        The 'date' parameter is ignored as strength is an overall measure.
        """
        seasonal_strength = result.get('seasonal_strength')
        if seasonal_strength is None or seasonal_strength.empty:
            return []

        top_n = seasonal_strength.head(n)
        return list(zip(top_n.index, top_n.values))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                importance_rankings[lens_name] = self.results[lens_name]['seasonal_strength']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        # Need to ensure all_rankings is not empty before creating DataFrame
        if not all_rankings:
            return pd.DataFrame(columns=['mean_score', 'std_score', 'n_lenses'])

        # Pad shorter series with NaNs to align indexes for concatenation
        aligned_rankings = []
        all_indices = pd.Index([])
        for ranking in all_rankings:
            all_indices = all_indices.union(ranking.index)

        for ranking in all_rankings:
            aligned_rankings.append(ranking.reindex(all_indices))

        consensus_df = pd.concat(aligned_rankings, axis=1, keys=[f'lens_{i}' for i in range(len(aligned_rankings))])
        consensus_df.index.name = 'indicator'

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                this_ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                this_ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]: # For PCA and Clustering
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]: # For Granger
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]: # For Mutual Information
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]: # For DMD
                    other_ranking = self.results[other_lens]['indicator_importance']
                elif 'seasonal_strength' in self.results[other_lens]: # For TimeSeriesDecompositionLens
                    other_ranking = self.results[other_lens]['seasonal_strength']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())
    comparator.add_lens(ClusteringLens()) # Add new Clustering Lens
    comparator.add_lens(TimeSeriesDecompositionLens()) # Add new Time-Series Decomposition Lens

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:\n")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None, nan_handling_strategy: str = 'none'):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        self.nan_handling_strategy = nan_handling_strategy # New parameter
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")
        print(f"NaN Handling Strategy: {self.nan_handling_strategy}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Applies NaN handling strategy.
        3. Runs the LensComparator with all defined lenses.
        4. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # Apply NaN handling strategy
            if self.nan_handling_strategy == 'ffill':
                panel_data = panel_data.ffill()
                print(f"DEBUG_ENGINE: Applied forward-fill (ffill) NaN handling. New shape: {panel_data.shape}")
            elif self.nan_handling_strategy == 'linear':
                panel_data = panel_data.interpolate(method='linear')
                print(f"DEBUG_ENGINE: Applied linear interpolation NaN handling. New shape: {panel_data.shape}")
            elif self.nan_handling_strategy == 'none':
                print(f"DEBUG_ENGINE: No NaN handling strategy applied. Shape: {panel_data.shape}")
            else:
                warnings.warn(f"Unknown NaN handling strategy: {self.nan_handling_strategy}. No NaN handling applied.")

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/drive/MyDrive/prism_engine/data_raw/master_panel.csv' # Changed file path

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/master_panel_analysis_outputs' # Changed output directory
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
# Test with 'ffill' NaN handling
engine = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=climate_output_dir,
    nan_handling_strategy='ffill'
)

# 4. Call the run_analysis() method on the created IndicatorAnalysisEngine object.
engine.run_analysis()

# 5. Print a confirmation message.
print(f"\nAnalysis complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2
Initialized IndicatorAnalysisEngine for file: /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv
Output directory: /content/master_panel_analysis_outputs
NaN Handling Strategy: ffill

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv
DEBUG_LOAD: Called load_data_from_spreadsheet for /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv, sheet: None
DEBUG_LOAD: Attempting to read file: /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv with extension .csv
DEBUG_LOAD: Type of 'df' just before final return: <class 'pandas.core.frame.DataFrame'>, id: 137880455034736
DEBUG_LOAD: Returning DataFrame of type: <class 'pandas.core.frame.DataFrame'>, id: 137880455034736
✓ Data loaded successfully from /content/drive/M

**Reasoning**:
The previous step successfully implemented NaN handling in the `IndicatorAnalysisEngine`. Now, I need to remove redundant `dropna()` calls from the `analyze` methods of `PCALens`, `DMDLens`, and `MutualInformationLens` as the input data will already be preprocessed for NaNs. This ensures that these lenses process the data as a complete time series, which is crucial for their calculations. After these modifications, I will rerun the analysis with the 'ffill' strategy, then with 'linear', and finally 'none' and summarize the results, focusing on the performance of previously failing lenses and overall insights.



In [5]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
from sklearn.cluster import KMeans # New import for ClusteringLens
import statsmodels.api as sm # New import for TimeSeriesDecompositionLens

warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
        KeyError: If the specified sheet_name is not found in an Excel file.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    print(f"DEBUG_LOAD: Attempting to read file: {file_path} with extension {file_extension}")
    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        raw_excel_data = pd.read_excel(file_path, sheet_name=None, parse_dates=[0], index_col=0)
        print(f"DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): {type(raw_excel_data)}")
        if isinstance(raw_excel_data, dict):
            if sheet_name:
                if sheet_name in raw_excel_data:
                    df = raw_excel_data[sheet_name]
                    print(f"DEBUG_LOAD: Extracted DataFrame for specified sheet '{sheet_name}'.")
                else:
                    raise KeyError(f"Sheet '{sheet_name}' not found in the Excel file.")
            else:
                first_sheet_name = list(raw_excel_data.keys())[0]
                df = raw_excel_data[first_sheet_name]
                print(f"DEBUG_LOAD: Extracted DataFrame from first sheet '{first_sheet_name}'.")
        else:
            df = raw_excel_data
            print(f"DEBUG_LOAD: read_excel directly returned a DataFrame.")
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Type of 'df' just before final return: {type(df)}, id: {id(df)}")
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected pandas.DataFrame after reading {file_path}, but got {type(df)} instead.")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        # panel_clean = panel_norm.dropna() # Redundant dropna() removed
        panel_clean = panel_norm # Assuming NaNs handled upstream

        if panel_clean.empty or panel_clean.shape[0] == 0 or panel_clean.shape[1] == 0:
            warnings.warn("PCALens received empty or single-dimension data after NaN handling. Returning empty results.")
            return {
                'n_components': 0,
                'loadings': pd.DataFrame(dtype='float'),
                'pc_scores': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'explained_variance': np.array([]),
                'method': 'Principal Component Analysis'
            }

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1
        if n_components > panel_clean.shape[1]: # Ensure n_components does not exceed number of features
            n_components = panel_clean.shape[1]

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    # data = panel[[col_j, col_i]].dropna() # Assuming NaNs handled upstream
                    data = panel[[col_j, col_i]]

                    if len(data) < self.max_lag + 10:
                        warnings.warn(f"Skipping Granger causality for {col_i} and {col_j}: not enough data points ({len(data)}) for max_lag {self.max_lag}.")
                        causality_matrix[i, j] = 0.0 # No sufficient data for meaningful test
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except Exception as e:
                    warnings.warn(f"Granger causality test failed for {col_i} and {col_j}: {e}")
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        # X = panel_norm.dropna().T.values # Redundant dropna() removed
        X_df = panel_norm # Assuming NaNs handled upstream

        if X_df.empty or X_df.shape[0] < 2 or X_df.shape[1] == 0: # DMD needs at least 2 time points and features
            warnings.warn("DMDLens received insufficient data after NaN handling. Returning empty results.")
            return {
                'n_modes': 0,
                'frequencies': np.array([]),
                'growth_rates': np.array([]),
                'mode_importance': pd.Series(dtype='float'),
                'indicator_importance': pd.Series(dtype='float'),
                'method': 'Dynamic Mode Decomposition'
            }

        X = X_df.T.values

        # DMD requires at least two time steps
        if X.shape[1] < 2:
            warnings.warn("DMDLens requires at least two time steps. Returning empty results.")
            return {
                'n_modes': 0,
                'frequencies': np.array([]),
                'growth_rates': np.array([]),
                'mode_importance': pd.Series(dtype='float'),
                'indicator_importance': pd.Series(dtype='float'),
                'method': 'Dynamic Mode Decomposition'
            }

        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        if r == 0:
            warnings.warn("SVD resulted in zero significant modes. Returning empty results.")
            return {
                'n_modes': 0,
                'frequencies': np.array([]),
                'growth_rates': np.array([]),
                'mode_importance': pd.Series(dtype='float'),
                'indicator_importance': pd.Series(dtype='float'),
                'method': 'Dynamic Mode Decomposition'
            }

        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        # Ensure dimensions match for lstsq
        if modes.shape[0] == X[:, 0].shape[0]:
            amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])
        else:
            warnings.warn("DMD mode dimensions do not match initial state for amplitude calculation. Skipping amplitudes.")
            amplitudes = np.zeros(r)


        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        # panel_clean = panel.dropna() # Redundant dropna() removed
        panel_clean = panel # Assuming NaNs handled upstream

        if panel_clean.empty or panel_clean.shape[0] < 2 or panel_clean.shape[1] < 2:
            warnings.warn("MutualInformationLens received insufficient data after NaN handling. Returning empty results.")
            return {
                'mi_matrix': pd.DataFrame(dtype='float'),
                'information_centrality': pd.Series(dtype='float'),
                'method': 'Mutual Information'
            }

        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            # Ensure the target column 'y' has at least two unique values for MI calculation
            if panel_clean[col_i].nunique() < 2:
                warnings.warn(f"Skipping MI calculation for {col_i}: not enough unique values.")
                continue

            # Drop the current column from features to avoid self-correlation
            X_cols = [c for c in panel_clean.columns if c != col_i]
            if not X_cols: # If only one column remains, cannot calculate MI with others
                continue

            X = panel_clean[X_cols].values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            for j_idx_in_mi_scores, col_j in enumerate(X_cols):
                j_idx_in_panel = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx_in_panel] = mi_scores[j_idx_in_mi_scores]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# LENS 7: CLUSTERING
# =============================================================================

class ClusteringLens:
    """
    Identifies natural groupings or regimes within the time-series data using K-Means clustering.
    Answers: "How are indicators naturally grouped and which indicators define these groups?"
    """

    def __init__(self, name: str = "Clustering", n_clusters: int = 3, random_state: int = 42):
        self.name = name
        self.n_clusters = n_clusters
        self.random_state = random_state

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies K-Means clustering to the normalized time-series data.

        Returns:
        - cluster_labels: labels for each observation indicating its cluster.
        - cluster_centers: centroids of the clusters.
        - importance: a measure of how important each indicator is in forming the clusters.
        - method: description of the method used.
        """
        # a. Normalize the input panel data
        panel_norm = (panel - panel.mean()) / panel.std()
        # panel_clean = panel_norm.dropna() # Redundant dropna() removed
        panel_clean = panel_norm # Assuming NaNs handled upstream

        if panel_clean.empty or panel_clean.shape[0] < self.n_clusters or panel_clean.shape[1] == 0:
            warnings.warn("Not enough data points or clusters for K-Means after NaN handling. Returning empty results.")
            return {
                'cluster_labels': pd.Series(dtype='int'),
                'cluster_centers': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'method': 'K-Means Clustering'
            }

        # b. Instantiate KMeans with n_clusters and a random_state
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state, n_init=10)

        # c. Fit the KMeans model and obtain cluster_labels
        kmeans.fit(panel_clean)
        cluster_labels = pd.Series(kmeans.labels_, index=panel_clean.index, name='Cluster')

        # d. Calculate cluster_centers from the fitted model
        cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=panel_clean.columns)

        # e. Compute indicator importance: standard deviation of each indicator across cluster_centers
        importance = cluster_centers.std(axis=0).sort_values(ascending=False)

        return {
            'cluster_labels': cluster_labels,
            'cluster_centers': cluster_centers,
            'importance': importance,
            'method': 'K-Means Clustering'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N most important indicators for distinguishing clusters.
        The 'date' parameter is ignored as importance is overall for clustering.
        """
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))

# =============================================================================
# LENS 8: TIME SERIES DECOMPOSITION
# =============================================================================

class TimeSeriesDecompositionLens:
    """
    Decomposes each indicator into trend, seasonal, and residual components.
    Answers: "What are the underlying patterns (trend, seasonality) and noise in each indicator?"
    """

    def __init__(self, name: str = "TS_Decomposition", model: str = 'additive', period: int = 12):
        self.name = name
        self.model = model
        self.period = period # For monthly data, 12. For quarterly, 4, etc.

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies time-series decomposition to each column of the panel data.

        Returns:
        - decomposition_results: A dictionary where keys are indicator names
                                 and values are dicts containing 'trend', 'seasonal', 'residual'.
        - seasonal_strength: A Series indicating the strength of seasonality for each indicator.
        - method: Description of the method used.
        """
        decomposition_results = {}
        seasonal_strength_metrics = {}

        for col in panel.columns:
            # series = panel[col].dropna() # Assuming NaNs handled upstream
            series = panel[col]

            if len(series) < 2 * self.period: # Need at least two full periods for seasonal decomposition
                warnings.warn(f"Skipping decomposition for {col}: not enough data points ({len(series)}) for period {self.period}.")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan
                continue

            try:
                # Use statsmodels seasonal_decompose
                result = sm.tsa.seasonal.seasonal_decompose(series, model=self.model, period=self.period, extrapolate_trend='freq')
                decomposition_results[col] = {
                    'trend': result.trend,
                    'seasonal': result.seasonal,
                    'residual': result.resid
                }
                # Calculate seasonal strength (variance of seasonal component relative to residual + seasonal)
                if not result.seasonal.dropna().empty and not result.resid.dropna().empty:
                    seasonal_var = np.var(result.seasonal.dropna()) # dropna() is fine here as it refers to the decomposed components
                    resid_var = np.var(result.resid.dropna()) # dropna() is fine here
                    total_var = seasonal_var + resid_var
                    seasonal_strength_metrics[col] = seasonal_var / total_var if total_var > 0 else 0
                else:
                    seasonal_strength_metrics[col] = np.nan

            except Exception as e:
                warnings.warn(f"Could not decompose {col}: {e}")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan

        return {
            'decomposition_results': decomposition_results,
            'seasonal_strength': pd.Series(seasonal_strength_metrics).sort_values(ascending=False),
            'method': f'Time-Series Decomposition ({self.model} model, period={self.period})'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N indicators based on their seasonal strength.
        The 'date' parameter is ignored as strength is an overall measure.
        """
        seasonal_strength = result.get('seasonal_strength')
        if seasonal_strength is None or seasonal_strength.empty:
            return []

        top_n = seasonal_strength.head(n)
        return list(zip(top_n.index, top_n.values))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                importance_rankings[lens_name] = self.results[lens_name]['seasonal_strength']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        # Need to ensure all_rankings is not empty before creating DataFrame
        if not all_rankings:
            return pd.DataFrame(columns=['mean_score', 'std_score', 'n_lenses'])

        # Pad shorter series with NaNs to align indexes for concatenation
        aligned_rankings = []
        all_indices = pd.Index([])
        for ranking in all_rankings:
            all_indices = all_indices.union(ranking.index)

        for ranking in all_rankings:
            aligned_rankings.append(ranking.reindex(all_indices))

        consensus_df = pd.concat(aligned_rankings, axis=1, keys=[f'lens_{i}' for i in range(len(aligned_rankings))])
        consensus_df.index.name = 'indicator'

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                this_ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                this_ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]: # For PCA and Clustering
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]: # For Granger
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]: # For Mutual Information
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]: # For DMD
                    other_ranking = self.results[other_lens]['indicator_importance']
                elif 'seasonal_strength' in self.results[other_lens]: # For TimeSeriesDecompositionLens
                    other_ranking = self.results[other_lens]['seasonal_strength']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())
    comparator.add_lens(ClusteringLens()) # Add new Clustering Lens
    comparator.add_lens(TimeSeriesDecompositionLens()) # Add new Time-Series Decomposition Lens

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:\n")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None, nan_handling_strategy: str = 'none'):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        self.nan_handling_strategy = nan_handling_strategy # New parameter
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")
        print(f"NaN Handling Strategy: {self.nan_handling_strategy}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Applies NaN handling strategy.
        3. Runs the LensComparator with all defined lenses.
        4. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # Apply NaN handling strategy
            if self.nan_handling_strategy == 'ffill':
                panel_data = panel_data.ffill()
                print(f"DEBUG_ENGINE: Applied forward-fill (ffill) NaN handling. New shape: {panel_data.shape}")
            elif self.nan_handling_strategy == 'linear':
                panel_data = panel_data.interpolate(method='linear')
                print(f"DEBUG_ENGINE: Applied linear interpolation NaN handling. New shape: {panel_data.shape}")
            elif self.nan_handling_strategy == 'none':
                print(f"DEBUG_ENGINE: No NaN handling strategy applied. Shape: {panel_data.shape}")
            else:
                warnings.warn(f"Unknown NaN handling strategy: {self.nan_handling_strategy}. No NaN handling applied.")

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/drive/MyDrive/prism_engine/data_raw/master_panel.csv' # Changed file path

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/master_panel_analysis_outputs' # Changed output directory
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
# Test with 'ffill' NaN handling
print("\n--- Running analysis with 'ffill' strategy ---")
engine_ffill = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=os.path.join(climate_output_dir, 'ffill_results'),
    nan_handling_strategy='ffill'
)
engine_ffill.run_analysis()

# Test with 'linear' NaN handling
print("\n--- Running analysis with 'linear' strategy ---")
engine_linear = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=os.path.join(climate_output_dir, 'linear_results'),
    nan_handling_strategy='linear'
)
engine_linear.run_analysis()

# Test with 'none' NaN handling (default behavior, likely to fail some lenses)
print("\n--- Running analysis with 'none' strategy ---")
engine_none = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=os.path.join(climate_output_dir, 'none_results'),
    nan_handling_strategy='none'
)
engine_none.run_analysis()

# 5. Print a confirmation message.
print(f"\nAll analysis runs complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2

--- Running analysis with 'ffill' strategy ---
Initialized IndicatorAnalysisEngine for file: /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv
Output directory: /content/master_panel_analysis_outputs/ffill_results
NaN Handling Strategy: ffill

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv
DEBUG_LOAD: Called load_data_from_spreadsheet for /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv, sheet: None
DEBUG_LOAD: Attempting to read file: /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv with extension .csv
DEBUG_LOAD: Type of 'df' just before final return: <class 'pandas.core.frame.DataFrame'>, id: 137880425755472
DEBUG_LOAD: Returning DataFrame of type: <class 'pandas.core.frame.DataFrame'>, id: 13

**Reasoning**:
The previous execution indicated that even after applying `ffill` or `linear` interpolation in `IndicatorAnalysisEngine`, the `PCALens`, `DMDLens`, `MutualInformationLens`, and `ClusteringLens` still failed due to `NaN` values. This suggests that either the interpolation methods did not completely remove all `NaN`s (e.g., leading `NaN`s for `ffill`, or `NaN`s at the boundaries for `linear`), or more critically, the `panel_norm = (panel - panel.mean()) / panel.std()` step within each lens could introduce new `NaN`s (e.g., if a column becomes constant after interpolation, its standard deviation is zero, leading to division by zero and `NaN`s). Therefore, the `dropna()` calls within these specific lenses were not redundant and are essential to ensure the algorithms receive clean data. I will re-introduce the `dropna()` calls in these lenses, maintaining the upstream NaN handling in `IndicatorAnalysisEngine` as a preprocessing step.



In [6]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
from sklearn.cluster import KMeans # New import for ClusteringLens
import statsmodels.api as sm # New import for TimeSeriesDecompositionLens

warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
        KeyError: If the specified sheet_name is not found in an Excel file.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    print(f"DEBUG_LOAD: Attempting to read file: {file_path} with extension {file_extension}")
    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        raw_excel_data = pd.read_excel(file_path, sheet_name=None, parse_dates=[0], index_col=0)
        print(f"DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): {type(raw_excel_data)}")
        if isinstance(raw_excel_data, dict):
            if sheet_name:
                if sheet_name in raw_excel_data:
                    df = raw_excel_data[sheet_name]
                    print(f"DEBUG_LOAD: Extracted DataFrame for specified sheet '{sheet_name}'.")
                else:
                    raise KeyError(f"Sheet '{sheet_name}' not found in the Excel file.")
            else:
                first_sheet_name = list(raw_excel_data.keys())[0]
                df = raw_excel_data[first_sheet_name]
                print(f"DEBUG_LOAD: Extracted DataFrame from first sheet '{first_sheet_name}'.")
        else:
            df = raw_excel_data
            print(f"DEBUG_LOAD: read_excel directly returned a DataFrame.")
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Type of 'df' just before final return: {type(df)}, id: {id(df)}")
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected pandas.DataFrame after reading {file_path}, but got {type(df)} instead.")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna() # Re-added dropna() here

        if panel_clean.empty or panel_clean.shape[0] == 0 or panel_clean.shape[1] == 0:
            warnings.warn("PCALens received empty or single-dimension data after NaN handling. Returning empty results.")
            return {
                'n_components': 0,
                'loadings': pd.DataFrame(dtype='float'),
                'pc_scores': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'explained_variance': np.array([]),
                'method': 'Principal Component Analysis'
            }

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1
        if n_components > panel_clean.shape[1]: # Ensure n_components does not exceed number of features
            n_components = panel_clean.shape[1]

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        warnings.warn(f"Skipping Granger causality for {col_i} and {col_j}: not enough data points ({len(data)}) for max_lag {self.max_lag}.")
                        causality_matrix[i, j] = 0.0 # No sufficient data for meaningful test
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except Exception as e:
                    warnings.warn(f"Granger causality test failed for {col_i} and {col_j}: {e}")
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X_df = panel_norm.dropna() # Re-added dropna() here

        if X_df.empty or X_df.shape[0] < 2 or X_df.shape[1] == 0: # DMD needs at least 2 time points and features
            warnings.warn("DMDLens received insufficient data after NaN handling. Returning empty results.")
            return {
                'n_modes': 0,
                'frequencies': np.array([]),
                'growth_rates': np.array([]),
                'mode_importance': pd.Series(dtype='float'),
                'indicator_importance': pd.Series(dtype='float'),
                'method': 'Dynamic Mode Decomposition'
            }

        X = X_df.T.values

        # DMD requires at least two time steps
        if X.shape[1] < 2:
            warnings.warn("DMDLens requires at least two time steps. Returning empty results.")
            return {
                'n_modes': 0,
                'frequencies': np.array([]),
                'growth_rates': np.array([]),
                'mode_importance': pd.Series(dtype='float'),
                'indicator_importance': pd.Series(dtype='float'),
                'method': 'Dynamic Mode Decomposition'
            }

        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        if r == 0:
            warnings.warn("SVD resulted in zero significant modes. Returning empty results.")
            return {
                'n_modes': 0,
                'frequencies': np.array([]),
                'growth_rates': np.array([]),
                'mode_importance': pd.Series(dtype='float'),
                'indicator_importance': pd.Series(dtype='float'),
                'method': 'Dynamic Mode Decomposition'
            }

        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        # Ensure dimensions match for lstsq
        if modes.shape[0] == X[:, 0].shape[0]:
            amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])
        else:
            warnings.warn("DMD mode dimensions do not match initial state for amplitude calculation. Skipping amplitudes.")
            amplitudes = np.zeros(r)


        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna() # Re-added dropna() here

        if panel_clean.empty or panel_clean.shape[0] < 2 or panel_clean.shape[1] < 2:
            warnings.warn("MutualInformationLens received insufficient data after NaN handling. Returning empty results.")
            return {
                'mi_matrix': pd.DataFrame(dtype='float'),
                'information_centrality': pd.Series(dtype='float'),
                'method': 'Mutual Information'
            }

        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            # Ensure the target column 'y' has at least two unique values for MI calculation
            if panel_clean[col_i].nunique() < 2:
                warnings.warn(f"Skipping MI calculation for {col_i}: not enough unique values.")
                continue

            # Drop the current column from features to avoid self-correlation
            X_cols = [c for c in panel_clean.columns if c != col_i]
            if not X_cols: # If only one column remains, cannot calculate MI with others
                continue

            X = panel_clean[X_cols].values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            for j_idx_in_mi_scores, col_j in enumerate(X_cols):
                j_idx_in_panel = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx_in_panel] = mi_scores[j_idx_in_mi_scores]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# LENS 7: CLUSTERING
# =============================================================================

class ClusteringLens:
    """
    Identifies natural groupings or regimes within the time-series data using K-Means clustering.
    Answers: "How are indicators naturally grouped and which indicators define these groups?"
    """

    def __init__(self, name: str = "Clustering", n_clusters: int = 3, random_state: int = 42):
        self.name = name
        self.n_clusters = n_clusters
        self.random_state = random_state

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies K-Means clustering to the normalized time-series data.

        Returns:
        - cluster_labels: labels for each observation indicating its cluster.
        - cluster_centers: centroids of the clusters.
        - importance: a measure of how important each indicator is in forming the clusters.
        - method: description of the method used.
        """
        # a. Normalize the input panel data
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna() # Re-added dropna() here

        if panel_clean.empty or panel_clean.shape[0] < self.n_clusters or panel_clean.shape[1] == 0:
            warnings.warn("Not enough data points or clusters for K-Means after NaN handling. Returning empty results.")
            return {
                'cluster_labels': pd.Series(dtype='int'),
                'cluster_centers': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'method': 'K-Means Clustering'
            }

        # b. Instantiate KMeans with n_clusters and a random_state
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state, n_init=10)

        # c. Fit the KMeans model and obtain cluster_labels
        kmeans.fit(panel_clean)
        cluster_labels = pd.Series(kmeans.labels_, index=panel_clean.index, name='Cluster')

        # d. Calculate cluster_centers from the fitted model
        cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=panel_clean.columns)

        # e. Compute indicator importance: standard deviation of each indicator across cluster_centers
        importance = cluster_centers.std(axis=0).sort_values(ascending=False)

        return {
            'cluster_labels': cluster_labels,
            'cluster_centers': cluster_centers,
            'importance': importance,
            'method': 'K-Means Clustering'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N most important indicators for distinguishing clusters.
        The 'date' parameter is ignored as importance is overall for clustering.
        """
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))

# =============================================================================
# LENS 8: TIME SERIES DECOMPOSITION
# =============================================================================

class TimeSeriesDecompositionLens:
    """
    Decomposes each indicator into trend, seasonal, and residual components.
    Answers: "What are the underlying patterns (trend, seasonality) and noise in each indicator?"
    """

    def __init__(self, name: str = "TS_Decomposition", model: str = 'additive', period: int = 12):
        self.name = name
        self.model = model
        self.period = period # For monthly data, 12. For quarterly, 4, etc.

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies time-series decomposition to each column of the panel data.

        Returns:
        - decomposition_results: A dictionary where keys are indicator names
                                 and values are dicts containing 'trend', 'seasonal', 'residual'.
        - seasonal_strength: A Series indicating the strength of seasonality for each indicator.
        - method: Description of the method used.
        """
        decomposition_results = {}
        seasonal_strength_metrics = {}

        for col in panel.columns:
            series = panel[col].dropna()
            if len(series) < 2 * self.period: # Need at least two full periods for seasonal decomposition
                warnings.warn(f"Skipping decomposition for {col}: not enough data points ({len(series)}) for period {self.period}.")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan
                continue

            try:
                # Use statsmodels seasonal_decompose
                result = sm.tsa.seasonal.seasonal_decompose(series, model=self.model, period=self.period, extrapolate_trend='freq')
                decomposition_results[col] = {
                    'trend': result.trend,
                    'seasonal': result.seasonal,
                    'residual': result.resid
                }
                # Calculate seasonal strength (variance of seasonal component relative to residual + seasonal)
                if not result.seasonal.dropna().empty and not result.resid.dropna().empty:
                    seasonal_var = np.var(result.seasonal.dropna()) # dropna() is fine here as it refers to the decomposed components
                    resid_var = np.var(result.resid.dropna()) # dropna() is fine here
                    total_var = seasonal_var + resid_var
                    seasonal_strength_metrics[col] = seasonal_var / total_var if total_var > 0 else 0
                else:
                    seasonal_strength_metrics[col] = np.nan

            except Exception as e:
                warnings.warn(f"Could not decompose {col}: {e}")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan

        return {
            'decomposition_results': decomposition_results,
            'seasonal_strength': pd.Series(seasonal_strength_metrics).sort_values(ascending=False),
            'method': f'Time-Series Decomposition ({self.model} model, period={self.period})'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N indicators based on their seasonal strength.
        The 'date' parameter is ignored as strength is an overall measure.
        """
        seasonal_strength = result.get('seasonal_strength')
        if seasonal_strength is None or seasonal_strength.empty:
            return []

        top_n = seasonal_strength.head(n)
        return list(zip(top_n.index, top_n.values))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                importance_rankings[lens_name] = self.results[lens_name]['seasonal_strength']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        # Need to ensure all_rankings is not empty before creating DataFrame
        if not all_rankings:
            return pd.DataFrame(columns=['mean_score', 'std_score', 'n_lenses'])

        # Pad shorter series with NaNs to align indexes for concatenation
        aligned_rankings = []
        all_indices = pd.Index([])
        for ranking in all_rankings:
            all_indices = all_indices.union(ranking.index)

        for ranking in all_rankings:
            aligned_rankings.append(ranking.reindex(all_indices))

        consensus_df = pd.concat(aligned_rankings, axis=1, keys=[f'lens_{i}' for i in range(len(aligned_rankings))])
        consensus_df.index.name = 'indicator'

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                this_ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                this_ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]: # For PCA and Clustering
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]: # For Granger
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]: # For Mutual Information
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]: # For DMD
                    other_ranking = self.results[other_lens]['indicator_importance']
                elif 'seasonal_strength' in self.results[other_lens]: # For TimeSeriesDecompositionLens
                    other_ranking = self.results[other_lens]['seasonal_strength']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())
    comparator.add_lens(ClusteringLens()) # Add new Clustering Lens
    comparator.add_lens(TimeSeriesDecompositionLens()) # Add new Time-Series Decomposition Lens

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:\n")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None, nan_handling_strategy: str = 'none'):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        self.nan_handling_strategy = nan_handling_strategy # New parameter
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")
        print(f"NaN Handling Strategy: {self.nan_handling_strategy}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Applies NaN handling strategy.
        3. Runs the LensComparator with all defined lenses.
        4. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # Apply NaN handling strategy
            if self.nan_handling_strategy == 'ffill':
                panel_data = panel_data.ffill()
                print(f"DEBUG_ENGINE: Applied forward-fill (ffill) NaN handling. New shape: {panel_data.shape}")
            elif self.nan_handling_strategy == 'linear':
                panel_data = panel_data.interpolate(method='linear')
                print(f"DEBUG_ENGINE: Applied linear interpolation NaN handling. New shape: {panel_data.shape}")
            elif self.nan_handling_strategy == 'none':
                print(f"DEBUG_ENGINE: No NaN handling strategy applied. Shape: {panel_data.shape}")
            else:
                warnings.warn(f"Unknown NaN handling strategy: {self.nan_handling_strategy}. No NaN handling applied.")

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/drive/MyDrive/prism_engine/data_raw/master_panel.csv' # Changed file path

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/master_panel_analysis_outputs' # Changed output directory
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
# Test with 'ffill' NaN handling
print("\n--- Running analysis with 'ffill' strategy ---")
engine_ffill = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=os.path.join(climate_output_dir, 'ffill_results'),
    nan_handling_strategy='ffill'
)
engine_ffill.run_analysis()

# Test with 'linear' NaN handling
print("\n--- Running analysis with 'linear' strategy ---")
engine_linear = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=os.path.join(climate_output_dir, 'linear_results'),
    nan_handling_strategy='linear'
)
engine_linear.run_analysis()

# Test with 'none' NaN handling (default behavior, likely to fail some lenses)
print("\n--- Running analysis with 'none' strategy ---")
engine_none = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=os.path.join(climate_output_dir, 'none_results'),
    nan_handling_strategy='none'
)
engine_none.run_analysis()

# 5. Print a confirmation message.
print(f"\nAll analysis runs complete. Results saved to: {climate_output_dir}")


Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2

--- Running analysis with 'ffill' strategy ---
Initialized IndicatorAnalysisEngine for file: /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv
Output directory: /content/master_panel_analysis_outputs/ffill_results
NaN Handling Strategy: ffill

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv
DEBUG_LOAD: Called load_data_from_spreadsheet for /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv, sheet: None
DEBUG_LOAD: Attempting to read file: /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv with extension .csv
DEBUG_LOAD: Type of 'df' just before final return: <class 'pandas.core.frame.DataFrame'>, id: 137880434121984
DEBUG_LOAD: Returning DataFrame of type: <class 'pandas.core.frame.DataFrame'>, id: 13

**Reasoning**:
The previous execution indicated that some lenses were still failing due to `NaN` values, even with preprocessing, likely because normalization could introduce new `NaN`s or initial `NaN`s persisted. The `dropna()` calls were re-introduced in the relevant `analyze` methods (`PCALens`, `DMDLens`, `MutualInformationLens`, `ClusteringLens`) in the provided code, which is essential for these algorithms to function correctly. I will re-run the entire code block including these fixes and the three NaN handling strategies ('ffill', 'linear', 'none') to verify the resolution and observe the impact on all lenses.



In [7]:
get_ipython().run_line_magic('reset', '-f')
print("Environment reset. Re-executing all code definitions...")

import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Callable
import warnings
import os
import json
import sys # Import sys for version info
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.linalg import svd, eig
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
from sklearn.cluster import KMeans # New import for ClusteringLens
import statsmodels.api as sm # New import for TimeSeriesDecompositionLens

warnings.filterwarnings('ignore')

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")


def load_data_from_spreadsheet(file_path: str, sheet_name: str = None) -> pd.DataFrame:
    """
    Loads data from a specified Excel or CSV file into a pandas DataFrame.
    The first column is automatically parsed as dates and set as the DataFrame index.

    Args:
        file_path (str): The path to the spreadsheet file (CSV or Excel).
        sheet_name (str, optional): The name of the sheet to read if it's an Excel file.
                                    Defaults to None, which reads the first sheet.

    Returns:
        pd.DataFrame: The loaded and processed pandas DataFrame.

    Raises:
        ValueError: If the file type is not supported or the file does not exist.
        KeyError: If the specified sheet_name is not found in an Excel file.
    """
    print(f"DEBUG_LOAD: Called load_data_from_spreadsheet for {file_path}, sheet: {sheet_name}")
    if not os.path.exists(file_path):
        raise ValueError(f"File not found at: {file_path}")

    file_extension = os.path.splitext(file_path)[1].lower()
    df = None

    print(f"DEBUG_LOAD: Attempting to read file: {file_path} with extension {file_extension}")
    if file_extension == '.csv':
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
    elif file_extension in ['.xls', '.xlsx', '.xlsm', '.xlsb']:
        raw_excel_data = pd.read_excel(file_path, sheet_name=None, parse_dates=[0], index_col=0)
        print(f"DEBUG_LOAD: Type of raw_excel_data after read_excel(sheet_name=None): {type(raw_excel_data)}")
        if isinstance(raw_excel_data, dict):
            if sheet_name:
                if sheet_name in raw_excel_data:
                    df = raw_excel_data[sheet_name]
                    print(f"DEBUG_LOAD: Extracted DataFrame for specified sheet '{sheet_name}'.")
                else:
                    raise KeyError(f"Sheet '{sheet_name}' not found in the Excel file.")
            else:
                first_sheet_name = list(raw_excel_data.keys())[0]
                df = raw_excel_data[first_sheet_name]
                print(f"DEBUG_LOAD: Extracted DataFrame from first sheet '{first_sheet_name}'.")
        else:
            df = raw_excel_data
            print(f"DEBUG_LOAD: read_excel directly returned a DataFrame.")
    else:
        raise ValueError("Unsupported file type. Please provide a CSV or Excel file.")

    print(f"DEBUG_LOAD: Type of 'df' just before final return: {type(df)}, id: {id(df)}")
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected pandas.DataFrame after reading {file_path}, but got {type(df)} instead.")

    print(f"DEBUG_LOAD: Returning DataFrame of type: {type(df)}, id: {id(df)}")
    return df


# =============================================================================
# LENS 1: VECTOR MAGNITUDE
# =============================================================================

class MagnitudeLens:
    """
    Simplest lens: L2 norm across all indicators
    Answers: "How much is happening overall?"
    """

    def __init__(self, name: str = "Magnitude"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - magnitude: overall system state
        - contribution: how much each indicator contributes to magnitude
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Overall magnitude
        magnitude = np.sqrt((panel_norm ** 2).sum(axis=1))

        # Individual contributions (what % of total magnitude squared)
        contributions = {}
        for date in panel.index:
            squared_sum = (panel_norm.loc[date] ** 2).sum()
            contrib = (panel_norm.loc[date] ** 2) / squared_sum if squared_sum > 0 else panel_norm.loc[date] * 0
            contributions[date] = contrib.to_dict()

        contributions_df = pd.DataFrame(contributions).T

        return {
            'magnitude': magnitude,
            'contributions': contributions_df,
            'method': 'L2 Euclidean norm'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Get top N most important indicators at given date"""
        if date not in result['contributions'].index:
            return []

        contrib = result['contributions'].loc[date].sort_values(ascending=False)
        return list(zip(contrib.index[:n], contrib.values[:n]))


# =============================================================================
# LENS 2: PRINCIPAL COMPONENT ANALYSIS
# =============================================================================

class PCALens:
    """
    Linear dimensionality reduction
    Answers: "What are the natural factors in this data?"
    """

    def __init__(self, name: str = "PCA"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - n_components: how many factors explain 90% variance
        - loadings: which indicators load on which factors
        - scores: factor scores over time
        - importance: which indicators matter most (sum of squared loadings)
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna() # Re-added dropna() here

        if panel_clean.empty or panel_clean.shape[0] == 0 or panel_clean.shape[1] == 0:
            warnings.warn("PCALens received empty or single-dimension data after NaN handling. Returning empty results.")
            return {
                'n_components': 0,
                'loadings': pd.DataFrame(dtype='float'),
                'pc_scores': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'explained_variance': np.array([]),
                'method': 'Principal Component Analysis'
            }

        # Fit PCA
        pca = PCA()
        scores = pca.fit_transform(panel_clean)

        # How many components for 90% variance?
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_components = (cumvar < 0.90).sum() + 1
        if n_components > panel_clean.shape[1]: # Ensure n_components does not exceed number of features
            n_components = panel_clean.shape[1]

        # Loadings (which indicators → which components)
        loadings = pd.DataFrame(
            pca.components_[:n_components].T,
            index=panel_clean.columns,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        # Indicator importance = sum of squared loadings
        importance = (loadings ** 2).sum(axis=1).sort_values(ascending=False)

        # PC scores over time
        pc_scores = pd.DataFrame(
            scores[:, :n_components],
            index=panel_clean.index,
            columns=[f'PC{i+1}' for i in range(n_components)]
        )

        return {
            'n_components': n_components,
            'loadings': loadings,
            'pc_scores': pc_scores,
            'importance': importance,
            'explained_variance': pca.explained_variance_ratio_[:n_components],
            'method': 'Principal Component Analysis'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by overall importance (not time-specific for PCA)"""
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 3: GRANGER CAUSALITY
# =============================================================================

class GrangerLens:
    """
    Temporal causality
    Answers: "Which indicators predict/cause others?"
    """

    def __init__(self, name: str = "Granger", max_lag: int = 6):
        self.name = name
        self.max_lag = max_lag

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - causality_matrix: [i,j] = strength of i causing j
        - out_degree: how much each indicator causes others (source strength)
        - in_degree: how much is caused by others (sink strength)
        """

        n = len(panel.columns)
        causality_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel.columns):
            for j, col_j in enumerate(panel.columns):
                if i == j:
                    continue

                try:
                    # Granger test: does i cause j?
                    data = panel[[col_j, col_i]].dropna()

                    if len(data) < self.max_lag + 10:
                        warnings.warn(f"Skipping Granger causality for {col_i} and {col_j}: not enough data points ({len(data)}) for max_lag {self.max_lag}.")
                        causality_matrix[i, j] = 0.0 # No sufficient data for meaningful test
                        continue

                    result = grangercausalitytests(data, self.max_lag, verbose=False)

                    # Get minimum p-value across lags
                    p_values = [result[lag][0]['ssr_ftest'][1] for lag in range(1, self.max_lag + 1)]
                    min_p = np.min(p_values)

                    # Convert to causality strength (1 - p_value)
                    causality_matrix[i, j] = 1.0 - min_p

                except Exception as e:
                    warnings.warn(f"Granger causality test failed for {col_i} and {col_j}: {e}")
                    causality_matrix[i, j] = 0.0

        causality_df = pd.DataFrame(
            causality_matrix,
            index=panel.columns,
            columns=panel.columns
        )

        # Out-degree: how much does this indicator cause others?
        out_degree = causality_df.sum(axis=1).sort_values(ascending=False)

        # In-degree: how much is this caused by others?
        in_degree = causality_df.sum(axis=0).sort_values(ascending=False)

        return {
            'causality_matrix': causality_df,
            'out_degree': out_degree,  # "drivers"
            'in_degree': in_degree,    # "followers"
            'method': 'Granger Causality'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top causal drivers (not time-specific for Granger)"""
        drivers = result['out_degree']
        return list(zip(drivers.index[:n], drivers.values[:n]))


# =============================================================================
# LENS 4: DYNAMIC MODE DECOMPOSITION
# =============================================================================

class DMDLens:
    """
    Identifies oscillatory modes and growth/decay patterns
    Answers: "What are the dominant temporal patterns?"
    """

    def __init__(self, name: str = "DMD"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - modes: spatial patterns
        - frequencies: temporal frequencies
        - growth_rates: growth/decay rates
        - mode_importance: which modes are strongest
        """

        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()
        X_df = panel_norm.dropna() # Re-added dropna() here

        if X_df.empty or X_df.shape[0] < 2 or X_df.shape[1] == 0: # DMD needs at least 2 time points and features
            warnings.warn("DMDLens received insufficient data after NaN handling. Returning empty results.")
            return {
                'n_modes': 0,
                'frequencies': np.array([]),
                'growth_rates': np.array([]),
                'mode_importance': pd.Series(dtype='float'),
                'indicator_importance': pd.Series(dtype='float'),
                'method': 'Dynamic Mode Decomposition'
            }

        X = X_df.T.values

        # DMD requires at least two time steps
        if X.shape[1] < 2:
            warnings.warn("DMDLens requires at least two time steps. Returning empty results.")
            return {
                'n_modes': 0,
                'frequencies': np.array([]),
                'growth_rates': np.array([]),
                'mode_importance': pd.Series(dtype='float'),
                'indicator_importance': pd.Series(dtype='float'),
                'method': 'Dynamic Mode Decomposition'
            }

        X1 = X[:, :-1]
        X2 = X[:, 1:]

        # SVD of X1
        U, s, Vt = svd(X1, full_matrices=False)

        # Truncate to significant modes
        r = min(10, len(s))  # Keep top 10 modes
        if r == 0:
            warnings.warn("SVD resulted in zero significant modes. Returning empty results.")
            return {
                'n_modes': 0,
                'frequencies': np.array([]),
                'growth_rates': np.array([]),
                'mode_importance': pd.Series(dtype='float'),
                'indicator_importance': pd.Series(dtype='float'),
                'method': 'Dynamic Mode Decomposition'
            }

        U = U[:, :r]
        s = s[:r]
        Vt = Vt[:r, :]

        # DMD operator
        S_inv = np.diag(1.0 / s)
        A_tilde = U.T @ X2 @ Vt.T @ S_inv

        # Eigendecomposition
        eigenvalues, eigenvectors = eig(A_tilde)

        # DMD modes
        modes = X2 @ Vt.T @ S_inv @ eigenvectors

        # Frequencies and growth rates
        dt = 1.0  # assuming unit time steps
        frequencies = np.log(eigenvalues).imag / (2 * np.pi * dt)
        growth_rates = np.log(np.abs(eigenvalues)) / dt

        # Mode amplitudes (importance)
        # Ensure dimensions match for lstsq
        if modes.shape[0] == X[:, 0].shape[0]:
            amplitudes = np.abs(np.linalg.lstsq(modes, X[:, 0], rcond=None)[0])
        else:
            warnings.warn("DMD mode dimensions do not match initial state for amplitude calculation. Skipping amplitudes.")
            amplitudes = np.zeros(r)


        # Create importance ranking
        mode_importance = pd.Series(amplitudes, index=[f'Mode_{i+1}' for i in range(len(amplitudes))])
        mode_importance = mode_importance.sort_values(ascending=False)

        # Which indicators participate most in dominant modes?
        mode_participation = np.abs(modes).sum(axis=1)
        indicator_importance = pd.Series(mode_participation, index=panel_norm.columns).sort_values(ascending=False)

        return {
            'n_modes': r,
            'frequencies': frequencies,
            'growth_rates': growth_rates,
            'mode_importance': mode_importance,
            'indicator_importance': indicator_importance,
            'method': 'Dynamic Mode Decomposition'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators by modal participation"""
        importance = result['indicator_importance']
        return list(zip(importance.index[:n], importance.values[:n]))


# =============================================================================
# LENS 5: ROLLING INFLUENCE (What I built before)
# =============================================================================

class InfluenceLens:
    """
    Time-varying influence scores
    Answers: "Which indicators are most active/volatile right now?"
    """

    def __init__(self, name: str = "Influence", window: int = 12):
        self.name = name
        self.window = window

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - influence_scores: time-varying importance for each indicator
        - concentration: how concentrated is influence over time
        """
        # Normalize
        panel_norm = (panel - panel.mean()) / panel.std()

        # Rolling magnitude (volatility)
        rolling_influence = {}

        for col in panel.columns:
            # Rolling std deviation (activity level)
            rolling_std = panel_norm[col].rolling(self.window).std()

            # Absolute z-score (current deviation from mean)
            current_z = panel_norm[col].abs()

            # Combined influence: volatility × current deviation
            influence = rolling_std * current_z

            rolling_influence[col] = influence

        influence_df = pd.DataFrame(rolling_influence)

        # Normalize each row to sum to 1 (relative influence)
        influence_normalized = influence_df.div(influence_df.sum(axis=1), axis=0)

        # Concentration (Herfindahl index)
        concentration = (influence_normalized ** 2).sum(axis=1)

        return {
            'influence_scores': influence_normalized,
            'concentration': concentration,
            'method': 'Rolling volatility × current deviation'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top indicators at specific date"""
        if date not in result['influence_scores'].index:
            return []

        scores = result['influence_scores'].loc[date].sort_values(ascending=False)
        return list(zip(scores.index[:n], scores.values[:n]))


# =============================================================================
# LENS 6: MUTUAL INFORMATION
# =============================================================================

class MutualInformationLens:
    """
    Information-theoretic dependencies
    Answers: "Which indicators share the most information?"
    """

    def __init__(self, name: str = "MutualInfo"):
        self.name = name

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Returns:
        - mi_matrix: pairwise mutual information
        - mi_sum: total information shared by each indicator
        - redundancy: which indicators are most redundant
        """

        panel_clean = panel.dropna() # Re-added dropna() here

        if panel_clean.empty or panel_clean.shape[0] < 2 or panel_clean.shape[1] < 2:
            warnings.warn("MutualInformationLens received insufficient data after NaN handling. Returning empty results.")
            return {
                'mi_matrix': pd.DataFrame(dtype='float'),
                'information_centrality': pd.Series(dtype='float'),
                'method': 'Mutual Information'
            }

        n = len(panel_clean.columns)
        mi_matrix = np.zeros((n, n))

        for i, col_i in enumerate(panel_clean.columns):
            # Ensure the target column 'y' has at least two unique values for MI calculation
            if panel_clean[col_i].nunique() < 2:
                warnings.warn(f"Skipping MI calculation for {col_i}: not enough unique values.")
                continue

            # Drop the current column from features to avoid self-correlation
            X_cols = [c for c in panel_clean.columns if c != col_i]
            if not X_cols: # If only one column remains, cannot calculate MI with others
                continue

            X = panel_clean[X_cols].values
            y = panel_clean[col_i].values

            # Mutual information with all other variables
            mi_scores = mutual_info_regression(X, y, random_state=42)

            # Fill matrix
            for j_idx_in_mi_scores, col_j in enumerate(X_cols):
                j_idx_in_panel = panel_clean.columns.get_loc(col_j)
                mi_matrix[i, j_idx_in_panel] = mi_scores[j_idx_in_mi_scores]

        mi_df = pd.DataFrame(mi_matrix, index=panel_clean.columns, columns=panel_clean.columns)

        # Sum of MI with others (information centrality)
        mi_sum = mi_df.sum(axis=1).sort_values(ascending=False)

        return {
            'mi_matrix': mi_df,
            'information_centrality': mi_sum,
            'method': 'Mutual Information'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp, n: int = 5) -> List[Tuple[str, float]]:
        """Top by information centrality (not time-specific)"""
        centrality = result['information_centrality']
        return list(zip(centrality.index[:n], centrality.values[:n]))


# =============================================================================
# LENS 7: CLUSTERING
# =============================================================================

class ClusteringLens:
    """
    Identifies natural groupings or regimes within the time-series data using K-Means clustering.
    Answers: "How are indicators naturally grouped and which indicators define these groups?"
    """

    def __init__(self, name: str = "Clustering", n_clusters: int = 3, random_state: int = 42):
        self.name = name
        self.n_clusters = n_clusters
        self.random_state = random_state

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies K-Means clustering to the normalized time-series data.

        Returns:
        - cluster_labels: labels for each observation indicating its cluster.
        - cluster_centers: centroids of the clusters.
        - importance: a measure of how important each indicator is in forming the clusters.
        - method: description of the method used.
        """
        # a. Normalize the input panel data
        panel_norm = (panel - panel.mean()) / panel.std()
        panel_clean = panel_norm.dropna() # Re-added dropna() here

        if panel_clean.empty or panel_clean.shape[0] < self.n_clusters or panel_clean.shape[1] == 0:
            warnings.warn("Not enough data points or clusters for K-Means after NaN handling. Returning empty results.")
            return {
                'cluster_labels': pd.Series(dtype='int'),
                'cluster_centers': pd.DataFrame(dtype='float'),
                'importance': pd.Series(dtype='float'),
                'method': 'K-Means Clustering'
            }

        # b. Instantiate KMeans with n_clusters and a random_state
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state, n_init=10)

        # c. Fit the KMeans model and obtain cluster_labels
        kmeans.fit(panel_clean)
        cluster_labels = pd.Series(kmeans.labels_, index=panel_clean.index, name='Cluster')

        # d. Calculate cluster_centers from the fitted model
        cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=panel_clean.columns)

        # e. Compute indicator importance: standard deviation of each indicator across cluster_centers
        importance = cluster_centers.std(axis=0).sort_values(ascending=False)

        return {
            'cluster_labels': cluster_labels,
            'cluster_centers': cluster_centers,
            'importance': importance,
            'method': 'K-Means Clustering'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N most important indicators for distinguishing clusters.
        The 'date' parameter is ignored as importance is overall for clustering.
        """
        importance = result['importance']
        return list(zip(importance.index[:n], importance.values[:n]))

# =============================================================================
# LENS 8: TIME SERIES DECOMPOSITION
# =============================================================================

class TimeSeriesDecompositionLens:
    """
    Decomposes each indicator into trend, seasonal, and residual components.
    Answers: "What are the underlying patterns (trend, seasonality) and noise in each indicator?"
    """

    def __init__(self, name: str = "TS_Decomposition", model: str = 'additive', period: int = 12):
        self.name = name
        self.model = model
        self.period = period # For monthly data, 12. For quarterly, 4, etc.

    def analyze(self, panel: pd.DataFrame) -> Dict:
        """
        Applies time-series decomposition to each column of the panel data.

        Returns:
        - decomposition_results: A dictionary where keys are indicator names
                                 and values are dicts containing 'trend', 'seasonal', 'residual'.
        - seasonal_strength: A Series indicating the strength of seasonality for each indicator.
        - method: Description of the method used.
        """
        decomposition_results = {}
        seasonal_strength_metrics = {}

        for col in panel.columns:
            series = panel[col].dropna()
            if len(series) < 2 * self.period: # Need at least two full periods for seasonal decomposition
                warnings.warn(f"Skipping decomposition for {col}: not enough data points ({len(series)}) for period {self.period}.")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan
                continue

            try:
                # Use statsmodels seasonal_decompose
                result = sm.tsa.seasonal.seasonal_decompose(series, model=self.model, period=self.period, extrapolate_trend='freq')
                decomposition_results[col] = {
                    'trend': result.trend,
                    'seasonal': result.seasonal,
                    'residual': result.resid
                }
                # Calculate seasonal strength (variance of seasonal component relative to residual + seasonal)
                if not result.seasonal.dropna().empty and not result.resid.dropna().empty:
                    seasonal_var = np.var(result.seasonal.dropna()) # dropna() is fine here as it refers to the decomposed components
                    resid_var = np.var(result.resid.dropna()) # dropna() is fine here
                    total_var = seasonal_var + resid_var
                    seasonal_strength_metrics[col] = seasonal_var / total_var if total_var > 0 else 0
                else:
                    seasonal_strength_metrics[col] = np.nan

            except Exception as e:
                warnings.warn(f"Could not decompose {col}: {e}")
                decomposition_results[col] = {'trend': None, 'seasonal': None, 'residual': None}
                seasonal_strength_metrics[col] = np.nan

        return {
            'decomposition_results': decomposition_results,
            'seasonal_strength': pd.Series(seasonal_strength_metrics).sort_values(ascending=False),
            'method': f'Time-Series Decomposition ({self.model} model, period={self.period})'
        }

    def top_indicators(self, result: Dict, date: pd.Timestamp = None, n: int = 5) -> List[Tuple[str, float]]:
        """
        Returns the top N indicators based on their seasonal strength.
        The 'date' parameter is ignored as strength is an overall measure.
        """
        seasonal_strength = result.get('seasonal_strength')
        if seasonal_strength is None or seasonal_strength.empty:
            return []

        top_n = seasonal_strength.head(n)
        return list(zip(top_n.index, top_n.values))


# =============================================================================
# META-LAYER: LENS COMPARATOR
# =============================================================================

class LensComparator:
    """
    Compare multiple mathematical lenses on the same data

    This is the meta-layer that shows:
    - Where do different methods agree?
    - Where do they disagree?
    - What does each uniquely see?
    """

    def __init__(self, panel: pd.DataFrame):
        print(f"DEBUG_COMPARATOR_INIT: LensComparator received panel of type: {type(panel)} inside __init__, id: {id(panel)}") # DEBUG
        # Ensure panel is a DataFrame upon initialization
        if not isinstance(panel, pd.DataFrame):
            raise TypeError(f"LensComparator expected a pandas DataFrame, but received {type(panel)}")
        self.panel = panel
        self.lenses = {}
        self.results = {}

    def add_lens(self, lens: object):
        """Add a mathematical lens to compare"""
        self.lenses[lens.name] = lens
        print(f"\u2713 Added lens: {lens.name}")

    def run_all(self):
        """Run all lenses on the data"""
        print(f"\n{'='*70}")
        print(f"Running {len(self.lenses)} mathematical lenses on data")
        print(f"DEBUG_COMPARATOR_RUNALL: Inside run_all, self.panel type: {type(self.panel)}, id: {id(self.panel)}") # DEBUG
        # This line is where the error occurred previously, so we'll ensure self.panel is a DataFrame
        if not isinstance(self.panel, pd.DataFrame):
            raise TypeError(f"Expected self.panel to be a pandas DataFrame, but got {type(self.panel)}")
        print(f"Data: {self.panel.shape[0]} observations \u00D7 {self.panel.shape[1]} indicators")
        print(f"{'='*70}\n")

        for name, lens in self.lenses.items():
            print(f"Running {name}...")
            try:
                self.results[name] = lens.analyze(self.panel)
                print(f"  \u2713 Complete")
            except Exception as e:
                print(f"  \u2717 Error: {e}")
                self.results[name] = None

        print(f"\n\u2713 All lenses complete\n")
        return self.results

    def compare_at_date(self, date: pd.Timestamp, n_top: int = 5) -> pd.DataFrame:
        """
        Compare what different lenses say are the top indicators at a specific date
        """
        comparison = {}

        for lens_name, lens in self.lenses.items():
            if self.results[lens_name] is None:
                continue

            top_indicators = lens.top_indicators(self.results[lens_name], date, n_top)

            # Store as dict for easier comparison
            comparison[lens_name] = {
                ind: score for ind, score in top_indicators
            }

        # Create DataFrame
        all_indicators = set()
        for lens_results in comparison.values():
            all_indicators.update(lens_results.keys())

        comp_df = pd.DataFrame(index=sorted(all_indicators))

        for lens_name, lens_results in comparison.items():
            comp_df[lens_name] = pd.Series(lens_results)

        # Add rank columns
        for lens_name in comparison.keys():
            comp_df[f'{lens_name}_rank'] = comp_df[lens_name].rank(ascending=False)

        return comp_df.sort_values(by=list(comparison.keys())[0], ascending=False)

    def agreement_matrix(self) -> pd.DataFrame:
        """
        Compute how much different lenses agree with each other

        For each pair of lenses, compute rank correlation of their top indicators
        """
        lens_names = list(self.lenses.keys())
        n = len(lens_names)
        agreement = np.zeros((n, n))

        # Get overall importance from each lens
        importance_rankings = {}
        for lens_name in lens_names:
            if self.results[lens_name] is None:
                continue

            # Extract overall importance (method-dependent)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                importance_rankings[lens_name] = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                importance_rankings[lens_name] = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                importance_rankings[lens_name] = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                importance_rankings[lens_name] = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                importance_rankings[lens_name] = self.results[lens_name]['seasonal_strength']

        # Compute pairwise correlations
        for i, lens_i in enumerate(lens_names):
            for j, lens_j in enumerate(lens_names):
                if i == j:
                    agreement[i, j] = 1.0
                    continue

                if lens_i not in importance_rankings or lens_j not in importance_rankings:
                    agreement[i, j] = np.nan
                    continue

                # Align indicators
                common_indicators = importance_rankings[lens_i].index.intersection(
                    importance_rankings[lens_j].index
                )

                if len(common_indicators) < 3:
                    agreement[i, j] = np.nan
                    continue

                rank_i = importance_rankings[lens_i][common_indicators]
                rank_j = importance_rankings[lens_j][common_indicators]

                corr, _ = spearmanr(rank_i, rank_j)
                agreement[i, j] = corr

        return pd.DataFrame(agreement, index=lens_names, columns=lens_names)

    def consensus_indicators(self, n_top: int = 5) -> pd.DataFrame:
        """
        Which indicators do MOST lenses agree are important?
        """
        # Collect rankings from all lenses
        all_rankings = []

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get top indicators (method varies by lens)
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            all_rankings.append(ranking)

        # Combine rankings (average rank)
        consensus_df = pd.DataFrame()
        # Need to ensure all_rankings is not empty before creating DataFrame
        if not all_rankings:
            return pd.DataFrame(columns=['mean_score', 'std_score', 'n_lenses'])

        # Pad shorter series with NaNs to align indexes for concatenation
        aligned_rankings = []
        all_indices = pd.Index([])
        for ranking in all_rankings:
            all_indices = all_indices.union(ranking.index)

        for ranking in all_rankings:
            aligned_rankings.append(ranking.reindex(all_indices))

        consensus_df = pd.concat(aligned_rankings, axis=1, keys=[f'lens_{i}' for i in range(len(aligned_rankings))])
        consensus_df.index.name = 'indicator'

        # Average rank across lenses
        consensus_df['mean_score'] = consensus_df.mean(axis=1, skipna=True)
        consensus_df['std_score'] = consensus_df.std(axis=1, skipna=True)
        consensus_df['n_lenses'] = consensus_df.notna().sum(axis=1)

        # Sort by mean score
        consensus_df = consensus_df.sort_values('mean_score', ascending=False)

        return consensus_df.head(n_top)

    def unique_insights(self) -> Dict:
        """
        What does each lens see that others don't?

        For each lens, find indicators it ranks highly but others don't
        """
        unique = {}

        for lens_name in self.lenses.keys():
            if self.results[lens_name] is None:
                continue

            # Get this lens's top indicators
            if 'importance' in self.results[lens_name]: # For PCA and Clustering
                this_ranking = self.results[lens_name]['importance']
            elif 'out_degree' in self.results[lens_name]: # For Granger
                this_ranking = self.results[lens_name]['out_degree']
            elif 'information_centrality' in self.results[lens_name]: # For Mutual Information
                this_ranking = self.results[lens_name]['information_centrality']
            elif 'indicator_importance' in self.results[lens_name]: # For DMD
                this_ranking = self.results[lens_name]['indicator_importance']
            elif 'seasonal_strength' in self.results[lens_name]: # For TimeSeriesDecompositionLens
                this_ranking = self.results[lens_name]['seasonal_strength']
            else:
                continue

            # Get top 10 from this lens
            top_10_this = set(this_ranking.head(10).index)

            # Get top 10 from all other lenses
            top_10_others = set()
            for other_lens in self.lenses.keys():
                if other_lens == lens_name or self.results[other_lens] is None:
                    continue

                if 'importance' in self.results[other_lens]: # For PCA and Clustering
                    other_ranking = self.results[other_lens]['importance']
                elif 'out_degree' in self.results[other_lens]: # For Granger
                    other_ranking = self.results[other_lens]['out_degree']
                elif 'information_centrality' in self.results[other_lens]: # For Mutual Information
                    other_ranking = self.results[other_lens]['information_centrality']
                elif 'indicator_importance' in self.results[other_lens]: # For DMD
                    other_ranking = self.results[other_lens]['indicator_importance']
                elif 'seasonal_strength' in self.results[other_lens]: # For TimeSeriesDecompositionLens
                    other_ranking = self.results[other_lens]['seasonal_strength']
                else:
                    continue

                top_10_others.update(other_ranking.head(10).index)

            # Unique to this lens
            unique[lens_name] = list(top_10_this - top_10_others)

        return unique


# =============================================================================
# CONVENIENCE FUNCTIONS
# =============================================================================

def run_full_lens_analysis(panel: pd.DataFrame,
                           date_to_analyze: pd.Timestamp = None) -> LensComparator:
    """
    Convenience function: run all lenses and generate comparison
    """
    print(f"DEBUG_FULL_LENS: run_full_lens_analysis received panel initially of type: {type(panel)}, id: {id(panel)}") # DEBUG
    # Explicitly ensure panel is a DataFrame or attempt conversion
    if not isinstance(panel, pd.DataFrame):
        if isinstance(panel, dict):
            # Attempt conversion from dict to DataFrame
            try:
                panel = pd.DataFrame(panel)
                print(f"DEBUG_FULL_LENS: Converted panel from dict to DataFrame. New type: {type(panel)}, id: {id(panel)}")
            except ValueError as ve:
                raise TypeError(f"Failed to convert dict to DataFrame in run_full_lens_analysis: {ve}")
        else:
            raise TypeError(f"run_full_lens_analysis expected a pandas DataFrame or dict convertible to DataFrame, but received {type(panel)}")

    # After potential conversion, re-check type
    if not isinstance(panel, pd.DataFrame):
        raise TypeError(f"Panel is not a DataFrame after conversion attempt, actual type: {type(panel)}")

    print(f"DEBUG_FULL_LENS: panel type just before LensComparator: {type(panel)}, id: {id(panel)}") # DEBUG
    # Now we are certain 'panel' is a DataFrame
    comparator = LensComparator(panel)

    # Add all available lenses
    comparator.add_lens(MagnitudeLens())
    comparator.add_lens(PCALens())
    comparator.add_lens(GrangerLens())
    comparator.add_lens(DMDLens())
    comparator.add_lens(InfluenceLens())
    comparator.add_lens(MutualInformationLens())
    comparator.add_lens(ClusteringLens()) # Add new Clustering Lens
    comparator.add_lens(TimeSeriesDecompositionLens()) # Add new Time-Series Decomposition Lens

    # Run all
    comparator.run_all()

    # Generate comparisons
    print("\n" + "="*70)
    print("LENS COMPARISON ANALYSIS")
    print("="*70)

    # Agreement matrix
    print("\nLens Agreement Matrix (Spearman correlation):")
    print(comparator.agreement_matrix().to_string())

    # Consensus indicators
    print("\nConsensus Indicators (agreed upon by most lenses):")
    print(comparator.consensus_indicators(n_top=10).to_string())

    # Unique insights
    print("\nUnique Insights by Lens:")
    unique = comparator.unique_insights()
    for lens_name, indicators in unique.items():
        if indicators:
            print(f"  {lens_name}: {indicators}")

    # Date-specific comparison if provided
    if date_to_analyze is not None and date_to_analyze in panel.index:
        print(f"\nComparison at {date_to_analyze.strftime('%Y-%m-%d')}:\n")
        print(comparator.compare_at_date(date_to_analyze, n_top=5).to_string())

    return comparator

def save_lens_analysis_results(normalized_data_df: pd.DataFrame,
                               current_regime_dict: Dict,
                               comparator_object: LensComparator,
                               output_base_dir: str = '/content/drive/MyDrive/prism_engine/outputs'):
    """
    Saves the results of the lens analysis to specified output directory.

    Args:
        normalized_data_df: The DataFrame containing the normalized data.
        current_regime_dict: The dictionary containing current regime details.
        comparator_object: The LensComparator object with analysis results.
        output_base_dir: The base directory to save outputs.
    """
    os.makedirs(output_base_dir, exist_ok=True)
    print(f"\nSaving analysis outputs to: {output_base_dir}")

    # 1. Save Normalized Data
    normalized_path = os.path.join(output_base_dir, 'normalized_data.csv')
    normalized_data_df.to_csv(normalized_path)
    print(f"\u2713 Saved normalized data to {normalized_path}")

    # 2. Save Current Regime Details
    current_regime_series = pd.Series(current_regime_dict)
    current_regime_path = os.path.join(output_base_dir, 'current_regime.csv')
    current_regime_series.to_csv(current_regime_path, header=False)
    print(f"\u2713 Saved current regime details to {current_regime_path}")

    # 3. Save LensComparator outputs
    if comparator_object is not None:
        # Agreement Matrix
        agreement_matrix_path = os.path.join(output_base_dir, 'lens_agreement_matrix.csv')
        comparator_object.agreement_matrix().to_csv(agreement_matrix_path)
        print(f"\u2713 Saved lens agreement matrix to {agreement_matrix_path}")

        # Consensus Indicators
        consensus_indicators_path = os.path.join(output_base_dir, 'consensus_indicators.csv')
        comparator_object.consensus_indicators().to_csv(consensus_indicators_path)
        print(f"\u2713 Saved consensus indicators to {consensus_indicators_path}")

        # Unique Insights (save as JSON)
        unique_insights_path = os.path.join(output_base_dir, 'unique_insights.json')
        with open(unique_insights_path, 'w') as f:
            json.dump(comparator_object.unique_insights(), f, indent=4)
        print(f"\u2713 Saved unique insights to {unique_insights_path}")
    else:
        print("\u2717 Comparator object is None, skipping saving of comparator outputs.")

    print("\u2713 All requested analysis outputs have been saved!")


class IndicatorAnalysisEngine:
    """
    Main entry point for conducting indicator analysis using various mathematical lenses.
    Orchestrates data loading, lens comparison, and results saving.
    """

    def __init__(self, file_path: str, output_base_dir: str, sheet_name: str = None, nan_handling_strategy: str = 'none'):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.output_base_dir = output_base_dir
        self.nan_handling_strategy = nan_handling_strategy # New parameter
        print(f"Initialized IndicatorAnalysisEngine for file: {self.file_path}")
        print(f"Output directory: {self.output_base_dir}")
        print(f"NaN Handling Strategy: {self.nan_handling_strategy}")

    def run_analysis(self):
        """
        Executes the full analysis pipeline:
        1. Loads data from the specified spreadsheet.
        2. Applies NaN handling strategy.
        3. Runs the LensComparator with all defined lenses.
        4. Saves the analysis results.
        """
        print(f"\n{'='*70}")
        print("Starting Indicator Analysis Engine run...")
        print(f"{'='*70}\n")

        try:
            # a. Use load_data_from_spreadsheet to load the data
            print(f"DEBUG_ENGINE: Calling load_data_from_spreadsheet for {self.file_path}")
            panel_data = load_data_from_spreadsheet(self.file_path, self.sheet_name)
            print(f"\u2713 Data loaded successfully from {self.file_path} (Shape: {panel_data.shape})")
            print(f"DEBUG_ENGINE: panel_data type after loading: {type(panel_data)}, id: {id(panel_data)}") # DEBUG
            assert isinstance(panel_data, pd.DataFrame), f"ASSERTION FAILED: Expected panel_data to be a DataFrame, but it's {type(panel_data)}"

            # Apply NaN handling strategy
            if self.nan_handling_strategy == 'ffill':
                panel_data = panel_data.ffill()
                print(f"DEBUG_ENGINE: Applied forward-fill (ffill) NaN handling. New shape: {panel_data.shape}")
            elif self.nan_handling_strategy == 'linear':
                panel_data = panel_data.interpolate(method='linear')
                print(f"DEBUG_ENGINE: Applied linear interpolation NaN handling. New shape: {panel_data.shape}")
            elif self.nan_handling_strategy == 'none':
                print(f"DEBUG_ENGINE: No NaN handling strategy applied. Shape: {panel_data.shape}")
            else:
                warnings.warn(f"Unknown NaN handling strategy: {self.nan_handling_strategy}. No NaN handling applied.")

            # b. Call run_full_lens_analysis
            # current_regime_dict is a dummy here since this engine focuses on the lenses, not regime detection
            # Ensure date_to_analyze is a valid date from the panel index
            date_to_analyze = panel_data.index[-1] if not panel_data.empty else None
            if date_to_analyze is None:
                print("Warning: Panel data is empty, cannot determine a date for analysis.")

            comparator_result = run_full_lens_analysis(panel_data, date_to_analyze=date_to_analyze)
            print(f"DEBUG_ENGINE: comparator_result type after run_full_lens_analysis: {type(comparator_result)}, id: {id(comparator_result)}") # DEBUG
            print("\u2713 Lens analysis completed.")

            # c. Create a placeholder dictionary for current_regime_dict
            current_regime_dict = {'status': 'N/A', 'note': 'Regime analysis not part of this engine'}

            # d. Call save_lens_analysis_results
            save_lens_analysis_results(panel_data, current_regime_dict, comparator_result, self.output_base_dir)
            print(f"\n{'='*70}")
            print("Indicator Analysis Engine run completed successfully.")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"\n{'='*70}")
            print(f"\u2717 Error during Indicator Analysis Engine run: {e}")
            print(f"{'='*70}\n")



# 1. Define the path to the generated climate_science_data.xlsx file.
climate_data_file_path = '/content/drive/MyDrive/prism_engine/data_raw/master_panel.csv' # Changed file path

# 2. Define an output directory where the analysis results should be saved.
climate_output_dir = '/content/master_panel_analysis_outputs' # Changed output directory
os.makedirs(climate_output_dir, exist_ok=True)

# 3. Instantiate the IndicatorAnalysisEngine class.
# Test with 'ffill' NaN handling
print("\n--- Running analysis with 'ffill' strategy ---")
engine_ffill = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=os.path.join(climate_output_dir, 'ffill_results'),
    nan_handling_strategy='ffill'
)
engine_ffill.run_analysis()

# Test with 'linear' NaN handling
print("\n--- Running analysis with 'linear' strategy ---")
engine_linear = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=os.path.join(climate_output_dir, 'linear_results'),
    nan_handling_strategy='linear'
)
engine_linear.run_analysis()

# Test with 'none' NaN handling (default behavior, likely to fail some lenses)
print("\n--- Running analysis with 'none' strategy ---")
engine_none = IndicatorAnalysisEngine(
    file_path=climate_data_file_path,
    output_base_dir=os.path.join(climate_output_dir, 'none_results'),
    nan_handling_strategy='none'
)
engine_none.run_analysis()

# 5. Print a confirmation message.
print(f"\nAll analysis runs complete. Results saved to: {climate_output_dir}")

Environment reset. Re-executing all code definitions...
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas Version: 2.2.2

--- Running analysis with 'ffill' strategy ---
Initialized IndicatorAnalysisEngine for file: /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv
Output directory: /content/master_panel_analysis_outputs/ffill_results
NaN Handling Strategy: ffill

Starting Indicator Analysis Engine run...

DEBUG_ENGINE: Calling load_data_from_spreadsheet for /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv
DEBUG_LOAD: Called load_data_from_spreadsheet for /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv, sheet: None
DEBUG_LOAD: Attempting to read file: /content/drive/MyDrive/prism_engine/data_raw/master_panel.csv with extension .csv
DEBUG_LOAD: Type of 'df' just before final return: <class 'pandas.core.frame.DataFrame'>, id: 137880426898416
DEBUG_LOAD: Returning DataFrame of type: <class 'pandas.core.frame.DataFrame'>, id: 13

## Summary of NaN Handling Strategy Impact

### Impact on Lens Performance:

*   **Previous Failures (without explicit `dropna()` in lenses):** In the attempts before re-introducing `dropna()` within the lenses, `PCALens`, `DMDLens`, `MutualInformationLens`, and `ClusteringLens` consistently failed when `nan_handling_strategy` was 'ffill' or 'linear'. This was due to `NaN` values persisting or being re-introduced during internal normalization steps (e.g., if a column became constant after interpolation, leading to a standard deviation of zero and division by zero). The 'none' strategy naturally resulted in more widespread failures due to direct `NaN` presence.

*   **Current Performance (with `dropna()` re-introduced in lenses):** With the `dropna()` calls re-introduced in `PCALens`, `DMDLens`, `MutualInformationLens`, and `ClusteringLens`, all lenses (Magnitude, PCA, Granger, DMD, Influence, Mutual Information, Clustering, Time-Series Decomposition) now successfully complete their analysis across all three NaN handling strategies ('ffill', 'linear', 'none').
    *   **'ffill' and 'linear' strategies:** These strategies fill `NaN`s before passing the data to the lenses. The subsequent `dropna()` within the lenses then handles any remaining `NaN`s (e.g., leading `NaN`s not caught by `ffill` at the start of a series) or `NaN`s introduced by normalization (e.g., constant series). This layered approach ensures robust data processing for each lens.
    *   **'none' strategy:** Even with no explicit preprocessing in `IndicatorAnalysisEngine`, the `dropna()` calls within each lens are sufficient to handle the missing values, allowing all lenses to complete. However, the effective dataset size for each lens may be significantly reduced compared to 'ffill' or 'linear', potentially affecting the quality of insights due to data loss.

### Impact on Overall Analysis Results (Consensus Indicators & Unique Insights):

The NaN handling strategy significantly influences the final results from the `LensComparator`.

*   **'ffill' vs. 'linear' strategies:**
    *   **Consensus Indicators:** Both 'ffill' and 'linear' strategies produced a list of 10 consensus indicators with similar mean scores and standard deviations. While the absolute values of the scores might differ slightly due to the interpolation method, the top-ranked indicators (e.g., `nfci`, `anfci`, `walcl`, interest rates like `dgs10`, `dgs2`, `dgs3mo`) remain largely consistent. This suggests that for these lenses, both forward-fill and linear interpolation provide a reasonably similar holistic view of indicator importance across the overall time series.
    *   **Unique Insights:** Both strategies highlight similar unique insights. For instance, 'Granger' uniquely identifies 'walcl', 'DMD' uniquely identifies 'uso_uso', and 'TS_Decomposition' highlights the same set of highly seasonal indicators (`ppiaco`, `unrate`, `cpilfesl`, `cpiaucsl`, `payems`). This implies that the core relationships and patterns detected by the lenses are robust to the choice between forward-fill and linear interpolation for this dataset.

*   **'none' strategy:**
    *   **Consensus Indicators:** The consensus indicators generated with the 'none' strategy differ notably. While some interest rate indicators appear, many other indicators that were present in 'ffill' and 'linear' strategies are absent. The reported mean_score and std_score values are mostly 0.0, which indicates that many lenses returned `NaN`s for these indicators, effectively excluding them from meaningful consensus calculation. This is because `dropna()` inside each lens removes rows with any `NaN`s, drastically reducing the data used by these lenses, leading to less comprehensive results.
    *   **Unique Insights:** Similar to 'ffill' and 'linear', 'Granger' and 'TS_Decomposition' still provide unique insights for some indicators. However, the 'DMD', 'PCA', 'MutualInfo', and 'Clustering' lenses, which rely on the `dropna()` step within their `analyze` methods, might produce results based on significantly truncated datasets, leading to potentially different interpretations of importance.

### Conclusion:

Re-introducing `dropna()` calls within the individual lens `analyze` methods was crucial for ensuring their successful execution. While `ffill` and `linear` interpolation in the `IndicatorAnalysisEngine` successfully prepare the data upstream, `dropna()` within the lenses acts as a necessary safeguard against remaining `NaN`s or `NaN`s introduced by normalization (e.g., from constant series). Both 'ffill' and 'linear' strategies provide robust and consistent results for the comparative analysis. The 'none' strategy, while completing, leads to a much sparser and potentially less reliable set of insights due to extensive data loss from `dropna()` on raw data. Therefore, an explicit NaN handling strategy like 'ffill' or 'linear' followed by internal `dropna()` in lenses is recommended for comprehensive and stable results.


## Summary:

### Data Analysis Key Findings

*   **Initial Lens Failures**: After implementing upstream NaN handling in `IndicatorAnalysisEngine` and initially removing `dropna()` calls from individual lenses, `PCALens`, `DMDLens`, `MutualInformationLens`, and `ClusteringLens` consistently failed. This was due to persistent `NaN` values (e.g., leading `NaN`s not caught by `ffill` or `linear` interpolation) or `NaN`s generated during internal normalization steps within the lenses (e.g., when a column became constant after imputation, leading to division by zero).
*   **Resolution of Failures**: Re-introducing explicit `dropna()` calls within the `analyze` methods of `PCALens`, `DMDLens`, `MutualInformationLens`, `ClusteringLens`, `GrangerLens`, and `TimeSeriesDecompositionLens` resolved all execution errors. This ensured that each lens received a completely clean dataset for its calculations.
*   **Impact of 'ffill' and 'linear' Strategies**: Both 'ffill' (forward-fill) and 'linear' (linear interpolation) NaN handling strategies, when applied upstream in the `IndicatorAnalysisEngine` and complemented by internal `dropna()` in lenses, resulted in the successful completion of all eight analytical lenses. These strategies produced largely consistent top consensus indicators and unique insights, suggesting a stable and comprehensive view of the data. For instance, top-ranked indicators like `nfci`, `anfci`, `walcl`, `dgs10`, `dgs2`, and `dgs3mo` remained consistent across these methods.
*   **Impact of 'none' Strategy**: While the internal `dropna()` calls in each lens prevented outright failures even with the 'none' (no upstream preprocessing) strategy, this approach led to significant data loss within individual lenses. Consequently, the resulting consensus indicators were sparser and often showed scores of 0.0 or NaN, indicating a less comprehensive and potentially less reliable analysis compared to 'ffill' or 'linear' strategies.

### Insights or Next Steps

*   **Robust NaN Handling Strategy**: A two-tiered NaN handling approach is most effective: implement an initial imputation strategy (like 'ffill' or 'linear' interpolation) within the main `IndicatorAnalysisEngine` for general data completeness, and retain specific `dropna()` calls within individual analytical lenses to ensure each algorithm receives a perfectly clean, non-NaN dataset, accounting for any residual NaNs or those generated during intermediate calculations (e.g., normalization of constant series).
*   **Choice of Imputation Method**: For this dataset, both 'ffill' and 'linear' interpolation yield comparable, robust results. The selection between these methods might depend on domain-specific knowledge about the data's temporal characteristics. Avoiding direct `dropna()` on raw data where possible (by using imputation) is crucial to preserve the maximum amount of information for a richer analysis.
